## Table of Contents
This notebook requires both a Python kernel and an R kernel
* [Preperation](#prep)
    * [Read in data](#data)
* [Sample description](#sample)
    * [Education and Age](#eduage)
    * [Parental disorder](#dis)
    * [Sample Comparison](#samplecom)
* [Frequentist Statistic](#frequentist)
    * [BoxPlot](#boxplot)
    * [KG vs. EG](#vs)
    * [Correlation parenting skill](#corr)
    * [Correlation parenting stress](#corrstress)
    * [Correlation Emotionregulation](#emotion)
    * [Correlation Heatmaps](#heat)
* [Bayesian Statistic - R Kernel](#bayes)
    * [Sample Comparison with R](#samplecomR)
    * [KG vs. EG with R](#bayeseg)
    * [Correlation parenting skill with R](#corrbayes)
    * [Correlation parenting stress with R](#corrstressbayes)
    * [Correlation Emotionregulation with R](#emotionbayes)


    bayeseg

### Preperation <a class="anchor" id="prep"></a>

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from neuroCombat import neuroCombat
from scipy.stats import norm
import scipy.stats as sc
import statsmodels

# base code
import numpy as np
import seaborn as sns
from statsmodels.tools.tools import maybe_unwrap_results
from statsmodels.graphics.gofplots import ProbPlot
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from typing import Type


class LinearRegDiagnostic():
    """
    Diagnostic plots to identify potential problems in a linear regression fit.
    Mainly,
        a. non-linearity of data
        b. Correlation of error terms
        c. non-constant variance
        d. outliers
        e. high-leverage points
        f. collinearity

    Authors:
        Prajwal Kafle (p33ajkafle@gmail.com, where 3 = r)
        Does not come with any sort of warranty.
        Please test the code one your end before using.

        Matt Spinelli (m3spinelli@gmail.com, where 3 = r)
        (1) Fixed incorrect annotation of the top most extreme residuals in
            the Residuals vs Fitted and, especially, the Normal Q-Q plots.
        (2) Changed Residuals vs Leverage plot to match closer the y-axis
            range shown in the equivalent plot in the R package ggfortify.
        (3) Added horizontal line at y=0 in Residuals vs Leverage plot to
            match the plots in R package ggfortify and base R.
        (4) Added option for placing a vertical guideline on the Residuals
            vs Leverage plot using the rule of thumb of h = 2p/n to denote
            high leverage (high_leverage_threshold=True).
        (5) Added two more ways to compute the Cook's Distance (D) threshold:
            * 'baseR': D > 1 and D > 0.5 (default)
            * 'convention': D > 4/n
            * 'dof': D > 4 / (n - k - 1)
        (6) Fixed class name to conform to Pascal casing convention
        (7) Fixed Residuals vs Leverage legend to work with loc='best'
    """

    def __init__(self,
                 results: Type[statsmodels.regression.linear_model.RegressionResultsWrapper]) -> None:
        """
        For a linear regression model, generates following diagnostic plots:

        a. residual
        b. qq
        c. scale location and
        d. leverage

        and a table

        e. vif

        Args:
            results (Type[statsmodels.regression.linear_model.RegressionResultsWrapper]):
                must be instance of statsmodels.regression.linear_model object

        Raises:
            TypeError: if instance does not belong to above object

        Example:
        >>> import numpy as np
        >>> import pandas as pd
        >>> import statsmodels.formula.api as smf
        >>> x = np.linspace(-np.pi, np.pi, 100)
        >>> y = 3*x + 8 + np.random.normal(0,1, 100)
        >>> df = pd.DataFrame({'x':x, 'y':y})
        >>> res = smf.ols(formula= "y ~ x", data=df).fit()
        >>> cls = Linear_Reg_Diagnostic(res)
        >>> cls(plot_context="seaborn-v0_8")

        In case you do not need all plots you can also independently make an individual plot/table
        in following ways

        >>> cls = Linear_Reg_Diagnostic(res)
        >>> cls.residual_plot()
        >>> cls.qq_plot()
        >>> cls.scale_location_plot()
        >>> cls.leverage_plot()
        >>> cls.vif_table()
        """

        if isinstance(results, statsmodels.regression.linear_model.RegressionResultsWrapper) is False:
            raise TypeError("result must be instance of statsmodels.regression.linear_model.RegressionResultsWrapper object")

        self.results = maybe_unwrap_results(results)

        self.y_true = self.results.model.endog
        self.y_predict = self.results.fittedvalues
        self.xvar = self.results.model.exog
        self.xvar_names = self.results.model.exog_names

        self.residual = np.array(self.results.resid)
        influence = self.results.get_influence()
        self.residual_norm = influence.resid_studentized_internal
        self.leverage = influence.hat_matrix_diag
        self.cooks_distance = influence.cooks_distance[0]
        self.nparams = len(self.results.params)
        self.nresids = len(self.residual_norm)

    def __call__(self, plot_context='seaborn-v0_8', **kwargs):
        # print(plt.style.available)
        # GH#9157
        if plot_context not in plt.style.available:
            plot_context = 'default'
        with plt.style.context(plot_context):
            fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
            self.residual_plot(ax=ax[0,0])
            self.qq_plot(ax=ax[0,1])
            self.scale_location_plot(ax=ax[1,0])
            self.leverage_plot(
                ax=ax[1,1],
                high_leverage_threshold = kwargs.get('high_leverage_threshold'),
                cooks_threshold = kwargs.get('cooks_threshold'))
            plt.show()

        return self.vif_table(), fig, ax,

    def residual_plot(self, ax=None):
        """
        Residual vs Fitted Plot

        Graphical tool to identify non-linearity.
        (Roughly) Horizontal red line is an indicator that the residual has a linear pattern
        """
        if ax is None:
            fig, ax = plt.subplots()

        sns.residplot(
            x=self.y_predict,
            y=self.residual,
            lowess=True,
            scatter_kws={'alpha': 0.5},
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
            ax=ax)

        # annotations
        residual_abs = np.abs(self.residual)
        abs_resid = np.flip(np.argsort(residual_abs), 0)
        abs_resid_top_3 = abs_resid[:3]
        for i in abs_resid_top_3:
            ax.annotate(
                i,
                xy=(self.y_predict[i], self.residual[i]),
                color='C3')

        ax.set_title('Residuals vs Fitted', fontweight="bold")
        ax.set_xlabel('Fitted values')
        ax.set_ylabel('Residuals')
        return ax

    def qq_plot(self, ax=None):
        """
        Standarized Residual vs Theoretical Quantile plot

        Used to visually check if residuals are normally distributed.
        Points spread along the diagonal line will suggest so.
        """
        if ax is None:
            fig, ax = plt.subplots()

        QQ = ProbPlot(self.residual_norm)
        fig = QQ.qqplot(line='45', alpha=0.5, lw=1, ax=ax)

        # annotations
        abs_norm_resid = np.flip(np.argsort(np.abs(self.residual_norm)), 0)
        abs_norm_resid_top_3 = abs_norm_resid[:3]
        for i, x, y in self.__qq_top_resid(QQ.theoretical_quantiles, abs_norm_resid_top_3):
            ax.annotate(
                i,
                xy=(x, y),
                ha='right',
                color='C3')

        ax.set_title('Normal Q-Q', fontweight="bold")
        ax.set_xlabel('Theoretical Quantiles')
        ax.set_ylabel('Standardized Residuals')
        return ax

    def scale_location_plot(self, ax=None):
        """
        Sqrt(Standarized Residual) vs Fitted values plot

        Used to check homoscedasticity of the residuals.
        Horizontal line will suggest so.
        """
        if ax is None:
            fig, ax = plt.subplots()

        residual_norm_abs_sqrt = np.sqrt(np.abs(self.residual_norm))

        ax.scatter(self.y_predict, residual_norm_abs_sqrt, alpha=0.5);
        sns.regplot(
            x=self.y_predict,
            y=residual_norm_abs_sqrt,
            scatter=False, ci=False,
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
            ax=ax)

        # annotations
        abs_sq_norm_resid = np.flip(np.argsort(residual_norm_abs_sqrt), 0)
        abs_sq_norm_resid_top_3 = abs_sq_norm_resid[:3]
        for i in abs_sq_norm_resid_top_3:
            ax.annotate(
                i,
                xy=(self.y_predict[i], residual_norm_abs_sqrt[i]),
                color='C3')

        ax.set_title('Scale-Location', fontweight="bold")
        ax.set_xlabel('Fitted values')
        ax.set_ylabel(r'$\sqrt{|\mathrm{Standardized\ Residuals}|}$');
        return ax

    def leverage_plot(self, ax=None, high_leverage_threshold=False, cooks_threshold='baseR'):
        """
        Residual vs Leverage plot

        Points falling outside Cook's distance curves are considered observation that can sway the fit
        aka are influential.
        Good to have none outside the curves.
        """
        if ax is None:
            fig, ax = plt.subplots()

        ax.scatter(
            self.leverage,
            self.residual_norm,
            alpha=0.5);

        sns.regplot(
            x=self.leverage,
            y=self.residual_norm,
            scatter=False,
            ci=False,
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
            ax=ax)

        # annotations
        leverage_top_3 = np.flip(np.argsort(self.cooks_distance), 0)[:3]
        for i in leverage_top_3:
            ax.annotate(
                i,
                xy=(self.leverage[i], self.residual_norm[i]),
                color = 'C3')

        factors = []
        if cooks_threshold == 'baseR' or cooks_threshold is None:
            factors = [1, 0.5]
        elif cooks_threshold == 'convention':
            factors = [4/self.nresids]
        elif cooks_threshold == 'dof':
            factors = [4/ (self.nresids - self.nparams)]
        else:
            raise ValueError("threshold_method must be one of the following: 'convention', 'dof', or 'baseR' (default)")
        for i, factor in enumerate(factors):
            label = "Cook's distance" if i == 0 else None
            xtemp, ytemp = self.__cooks_dist_line(factor)
            ax.plot(xtemp, ytemp, label=label, lw=1.25, ls='--', color='red')
            ax.plot(xtemp, np.negative(ytemp), lw=1.25, ls='--', color='red')

        if high_leverage_threshold:
            high_leverage = 2 * self.nparams / self.nresids
            if max(self.leverage) > high_leverage:
                ax.axvline(high_leverage, label='High leverage', ls='-.', color='purple', lw=1)

        ax.axhline(0, ls='dotted', color='black', lw=1.25)
        ax.set_xlim(0, max(self.leverage)+0.01)
        ax.set_ylim(min(self.residual_norm)-0.1, max(self.residual_norm)+0.1)
        ax.set_title('Residuals vs Leverage', fontweight="bold")
        ax.set_xlabel('Leverage')
        ax.set_ylabel('Standardized Residuals')
        plt.legend(loc='best')
        return ax

    def vif_table(self):
        """
        VIF table

        VIF, the variance inflation factor, is a measure of multicollinearity.
        VIF > 5 for a variable indicates that it is highly collinear with the
        other input variables.
        """
        vif_df = pd.DataFrame()
        vif_df["Features"] = self.xvar_names
        vif_df["VIF Factor"] = [variance_inflation_factor(self.xvar, i) for i in range(self.xvar.shape[1])]

        return (vif_df
                .sort_values("VIF Factor")
                .round(2))


    def __cooks_dist_line(self, factor):
        """
        Helper function for plotting Cook's distance curves
        """
        p = self.nparams
        formula = lambda x: np.sqrt((factor * p * (1 - x)) / x)
        x = np.linspace(0.001, max(self.leverage), 50)
        y = formula(x)
        return x, y


    def __qq_top_resid(self, quantiles, top_residual_indices):
        """
        Helper generator function yielding the index and coordinates
        """
        offset = 0
        quant_index = 0
        previous_is_negative = None
        for resid_index in top_residual_indices:
            y = self.residual_norm[resid_index]
            is_negative = y < 0
            if previous_is_negative == None or previous_is_negative == is_negative:
                offset += 1
            else:
                quant_index -= offset
            x = quantiles[quant_index] if is_negative else np.flip(quantiles, 0)[quant_index]
            quant_index += 1
            previous_is_negative = is_negative
            yield resid_index, x, y

def show_missing(df):
    """Return a Pandas dataframe describing the contents of a source dataframe including missing values."""
    pd.options.mode.use_inf_as_na = True   #to see inf values as NaN
    variables = []
    dtypes = []
    count = []
    missing = []
    pc_missing = []

    for item in df.columns:
        variables.append(item)
        dtypes.append(df[item].dtype)
        count.append(len(df[item]))
        missing.append(df[item].isna().sum())
        pc_missing.append(round((df[item].isna().sum() / len(df[item])) * 100, 2))

    output = pd.DataFrame({
    'variable': variables, 
    'dtype': dtypes,
    'count': count,
    'missing': missing, 
    'pc_missing': pc_missing
    })    

    print(output.loc[output['missing'] > 1])


#### Read in data <a class="anchor" id="data"></a>

In [ ]:
path='' #set path to data

#Read in data
stats_file = os.path.join(path + 'data.csv')
df = pd.read_csv(stats_file,sep=',', decimal=".", encoding='utf-8')
df = df.replace(' ', np.nan)# set empty values to NaN, cause they missing
df['Kind1_CBCL.Tsum'] = pd.to_numeric(df['Kind1_CBCL.Tsum'],errors='coerce')#NaN ingore 
final=df 
final = final.replace(['EG','KG'],[1.0,0.0]) #EG=1 KG=0

#Split into F3 / F4 disorders
#F4 = final[(final['Patient_PrimDiagnoseT1.ICD10'] == 'F41.1') | (final['Patient_PrimDiagnoseT1.ICD10'] == 'F43.21') | (final['Patient_PrimDiagnoseT1.ICD10'] == 'F45.1') | (final['Patient_PrimDiagnoseT1.ICD10'] == 'F43.10') | (final['Patient_PrimDiagnoseT1.ICD10'] == 'F41.0')| (final['Patient_PrimDiagnoseT1.ICD10'] == 'F40.218')| (final['Patient_PrimDiagnoseT1.ICD10'] == 'F43.23') | (final['Patient_PrimDiagnoseT1.ICD10'].isna())]
#F3 = final[(final['Patient_PrimDiagnoseT1.ICD10'] == 'F34.1') | (final['Patient_PrimDiagnoseT1.ICD10'] == 'F33.1') | (final['Patient_PrimDiagnoseT1.ICD10'] == 'F32.0') | (final['Patient_PrimDiagnoseT1.ICD10'] == 'F33.41') | (final['Patient_PrimDiagnoseT1.ICD10'] == 'F32.4')| (final['Patient_PrimDiagnoseT1.ICD10'] == 'F34.1')| (final['Patient_PrimDiagnoseT1.ICD10'].isna())]


#Take randomised one sibling out of the sample
#import random
#df_Gesch= df[df["Geschwisterkind"]=="ja"]
#df_ohne = df[df["Geschwisterkind"]=="nein"]

#91034 #91036 only one number exist as the other sibling didn´t made it trough quality control
#df_ohne = pd.concat([df_ohne, df[df["famID"] == 91034]])
#df_ohne = pd.concat([df_ohne, df[df["famID"] == 91036]])

#ID = ('3060',"3007","1014","91029","91032","93020","91059","93101","3030")
#one_sib_total = pd.DataFrame()
#random.seed(10)
#for sibling in ID:   
#    test = df_Gesch[df_Gesch["famID"] == int(sibling)]
#    one_sib = test.iloc[[random.randint(0,1)]]
#    one_sib_total = pd.concat([one_sib_total,one_sib])
#df_onesib = pd.concat([one_sib_total,df_ohne])  

#z-transform the numeric data   
    #df = full sample
    #zdf  = full sample z-transformed
    #zdf_onesib = randomised take one sibling z-transformed
    #zdf_quest =  sample where all subjects with missing on questionnaire are removed z-transformed  
zdf=df.copy()
numeric_cols = zdf.select_dtypes(include=[np.number]).columns 
zdf[numeric_cols] = zdf[numeric_cols].apply(sc.zscore,nan_policy='omit')


#zdf_onesib = df_onesib.copy()
#numeric_cols = zdf_onesib.select_dtypes(include=[np.number]).columns 
#zdf_onesib[numeric_cols] = zdf_onesib[numeric_cols].apply(sc.zscore,nan_policy='omit')

zdf_quest = zdf[zdf['Kind1_ESF.ES'].notna() & zdf['Kind1_FEEL_KJ.selbst_maladaptiv_gesamt'].notna() & zdf['Kind1_EFB.WS'].notna() & zdf["Kind1_FEEL_KJ.fremd_adaptiv_gesamt"]]
df_quest = df[df['Kind1_ESF.ES'].notna() & df['Kind1_FEEL_KJ.selbst_maladaptiv_gesamt'].notna() & df['Kind1_EFB.WS'].notna() & df["Kind1_FEEL_KJ.fremd_adaptiv_gesamt"]]

### Sample description <a class="anchor" id="sample"></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore') #deactivate warnings 
#Sample description, Anzahl, Alter, Geschlecht, Geschwister, CBCL.Tsum
print('Anzahl EG: ' + str(df[df['Gruppenzugehörigkeit']=='EG'].shape[0]) + '\nMittleres Alter: ' + str(round(df['AlterJahreMonate'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['AlterJahreMonate'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')'+ '\nGeschlecht: ' + str(df[df['Gruppenzugehörigkeit']=='EG'][df['Geschlecht']=='weiblich'].shape[0])+' weiblich und '+ str(df[df['Gruppenzugehörigkeit']=='EG'][df['Geschlecht']=='männlich'].shape[0]) + ' männlich'+ ' \nGeschwister: ' + str(df[df['Geschwisterkind']=='ja'][df['Gruppenzugehörigkeit']=='EG'].shape[0])+ ' (4 Geschwisterpaare 2*4) \nCBCL T-Wert Gesamtskala: ' + str(round(df['Kind1_CBCL.Tsum'][df['Gruppenzugehörigkeit']=='EG'].mean(skipna=True),3))+ ' (std: ' +str(round(df['Kind1_CBCL.Tsum'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+')'+ '\nSES Gesamtskala: ' + str(round(df['SES_2'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['SES_2'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')      
print('\n\nAnzahl KG: ' + str(df[df['Gruppenzugehörigkeit']=='KG'].shape[0]) + '\nMittleres Alter: ' + str(round(df['AlterJahreMonate'][df['Gruppenzugehörigkeit']=='KG'].mean(),3)) + ' (std: ' +str(round(df['AlterJahreMonate'][df['Gruppenzugehörigkeit']=='KG'].std(),3)) + ')' + '\nGeschlecht: ' + str(df[df['Gruppenzugehörigkeit']=='KG'][df['Geschlecht']=='weiblich'].shape[0])+' weiblich und ' + str(df[df['Gruppenzugehörigkeit']=='KG'][df['Geschlecht']=='männlich'].shape[0]) + ' männlich' + '\nGeschwister: ' + str(df[df['Geschwisterkind']=='ja'][df['Gruppenzugehörigkeit']=='KG'].shape[0]) + ' (5 Geschwisterpaare 2*5)' '\nCBCL T-Wert Gesamtskala: ' + str(round(df['Kind1_CBCL.Tsum'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_CBCL.Tsum'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ '\nSES Gesamtskala: ' + str(round(df['SES_2'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['SES_2'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')\n\n')
print("Tage zwischen MRT und Labor: \nmedian: " +str(df['daysbetweenmriandlabor'].median()) + ' \nmin Tage: '+str(df['daysbetweenmriandlabor'].min()) + '\nmax Tage: '+str(df['daysbetweenmriandlabor'].max()))
print('\nAnzahl erhobenenr Probanden Gießen: ' + str(df[df['Standort']=='Gießen'].shape[0]))
print('Anzahl erhobenenr Probanden Dortmund: ' + str(df[df['Standort']=='Dortmund'].shape[0]))  

#check which dataframe 
Zwischenspeicher = df.copy()
df = df_quest
print('\nEG = ' +  str(df[df['Gruppenzugehörigkeit']=='EG'].shape[0]), ' M(std):')
print('Kind1_FEEL_KJ.fremd_maladaptiv_gesamt: ' + str(round(df['Kind1_FEEL_KJ.fremd_maladaptiv_gesamt'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_FEEL_KJ.fremd_maladaptiv_gesamt'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('Kind1_FEEL_KJ.fremd_adaptiv_gesamt: ' + str(round(df['Kind1_FEEL_KJ.fremd_adaptiv_gesamt'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_FEEL_KJ.fremd_adaptiv_gesamt'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('Kind1_FEEL_KJ.selbst_maladaptiv_gesamt: ' + str(round(df['Kind1_FEEL_KJ.selbst_maladaptiv_gesamt'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_FEEL_KJ.selbst_maladaptiv_gesamt'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('Kind1_FEEL_KJ.selbst_adaptiv_gesamt: ' + str(round(df['Kind1_FEEL_KJ.selbst_adaptiv_gesamt'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_FEEL_KJ.selbst_adaptiv_gesamt'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('Kind1_EFB.GW: ' + str(round(df['Kind1_EFB.GW'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_EFB.GW'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('Kind1_EFB.NS: ' + str(round(df['Kind1_EFB.NS'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_EFB.NS'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('Kind1_EFB.UR: ' + str(round(df['Kind1_EFB.UR'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_EFB.UR'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('Kind1_EFB.WS: ' + str(round(df['Kind1_EFB.WS'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_EFB.WS'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('Kind1_ESF.ES: ' + str(round(df['Kind1_ESF.ES'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_ESF.ES'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('Kind1_ESF.RR: ' + str(round(df['Kind1_ESF.RR'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_ESF.RR'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('Kind1_ESF.SU: ' + str(round(df['Kind1_ESF.SU'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_ESF.SU'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('Kind1_ESF.PS: ' + str(round(df['Kind1_ESF.PS'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_ESF.PS'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('AlterJahreMonate: ' + str(round(df['AlterJahreMonate'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['AlterJahreMonate'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('SES_2: ' + str(round(df['SES_2'][df['Gruppenzugehörigkeit']=='EG'].mean(),3))+ ' (std: ' +str(round(df['SES_2'][df['Gruppenzugehörigkeit']=='EG'].std(),3))+ ')')
print('Geschlecht: ' + str(df[df['Gruppenzugehörigkeit']=='EG'][df['Geschlecht']=='weiblich'].shape[0])+' weiblich und '+ str(df[df['Gruppenzugehörigkeit']=='EG'][df['Geschlecht']=='männlich'].shape[0]) + ' männlich')
      
print('\nKG = ' +  str(df[df['Gruppenzugehörigkeit']=='KG'].shape[0]), ' M(std):')
print('Kind1_FEEL_KJ.fremd_maladaptiv_gesamt: ' + str(round(df['Kind1_FEEL_KJ.fremd_maladaptiv_gesamt'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_FEEL_KJ.fremd_maladaptiv_gesamt'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('Kind1_FEEL_KJ.fremd_adaptiv_gesamt: ' + str(round(df['Kind1_FEEL_KJ.fremd_adaptiv_gesamt'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_FEEL_KJ.fremd_adaptiv_gesamt'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('Kind1_FEEL_KJ.selbst_maladaptiv_gesamt: ' + str(round(df['Kind1_FEEL_KJ.selbst_maladaptiv_gesamt'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_FEEL_KJ.selbst_maladaptiv_gesamt'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('Kind1_FEEL_KJ.selbst_adaptiv_gesamt: ' + str(round(df['Kind1_FEEL_KJ.selbst_adaptiv_gesamt'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_FEEL_KJ.selbst_adaptiv_gesamt'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('Kind1_EFB.GW: ' + str(round(df['Kind1_EFB.GW'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_EFB.GW'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('Kind1_EFB.NS: ' + str(round(df['Kind1_EFB.NS'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_EFB.NS'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('Kind1_EFB.UR: ' + str(round(df['Kind1_EFB.UR'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_EFB.UR'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('Kind1_EFB.WS: ' + str(round(df['Kind1_EFB.WS'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_EFB.WS'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('Kind1_ESF.ES: ' + str(round(df['Kind1_ESF.ES'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_ESF.ES'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('Kind1_ESF.RR: ' + str(round(df['Kind1_ESF.RR'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_ESF.RR'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('Kind1_ESF.SU: ' + str(round(df['Kind1_ESF.SU'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_ESF.SU'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('Kind1_ESF.PS: ' + str(round(df['Kind1_ESF.PS'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['Kind1_ESF.PS'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('AlterJahreMonate: ' + str(round(df['AlterJahreMonate'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['AlterJahreMonate'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('SES_2: ' + str(round(df['SES_2'][df['Gruppenzugehörigkeit']=='KG'].mean(),3))+ ' (std: ' +str(round(df['SES_2'][df['Gruppenzugehörigkeit']=='KG'].std(),3))+ ')')
print('Geschlecht: ' + str(df[df['Gruppenzugehörigkeit']=='KG'][df['Geschlecht']=='weiblich'].shape[0])+' weiblich und '+ str(df[df['Gruppenzugehörigkeit']=='KG'][df['Geschlecht']=='männlich'].shape[0]) + ' männlich')

print('\nFull Sample = ' +  str(df.shape[0]), ' M(std):')
print('Kind1_FEEL_KJ.fremd_maladaptiv_gesamt: ' + str(round(df['Kind1_FEEL_KJ.fremd_maladaptiv_gesamt'].mean(),3))+ ' (std: ' +str(round(df['Kind1_FEEL_KJ.fremd_maladaptiv_gesamt'].std(),3))+ ')')
print('Kind1_FEEL_KJ.fremd_adaptiv_gesamt: ' + str(round(df['Kind1_FEEL_KJ.fremd_adaptiv_gesamt'].mean(),3))+ ' (std: ' +str(round(df['Kind1_FEEL_KJ.fremd_adaptiv_gesamt'].std(),3))+ ')')
print('Kind1_FEEL_KJ.selbst_maladaptiv_gesamt: ' + str(round(df['Kind1_FEEL_KJ.selbst_maladaptiv_gesamt'].mean(),3))+ ' (std: ' +str(round(df['Kind1_FEEL_KJ.selbst_maladaptiv_gesamt'].std(),3))+ ')')
print('Kind1_FEEL_KJ.selbst_adaptiv_gesamt: ' + str(round(df['Kind1_FEEL_KJ.selbst_adaptiv_gesamt'].mean(),3))+ ' (std: ' +str(round(df['Kind1_FEEL_KJ.selbst_adaptiv_gesamt'].std(),3))+ ')')
print('Kind1_EFB.GW: ' + str(round(df['Kind1_EFB.GW'].mean(),3))+ ' (std: ' +str(round(df['Kind1_EFB.GW'].std(),3))+ ')')
print('Kind1_EFB.NS: ' + str(round(df['Kind1_EFB.NS'].mean(),3))+ ' (std: ' +str(round(df['Kind1_EFB.NS'].std(),3))+ ')')
print('Kind1_EFB.UR: ' + str(round(df['Kind1_EFB.UR'].mean(),3))+ ' (std: ' +str(round(df['Kind1_EFB.UR'].std(),3))+ ')')
print('Kind1_EFB.WS: ' + str(round(df['Kind1_EFB.WS'].mean(),3))+ ' (std: ' +str(round(df['Kind1_EFB.WS'].std(),3))+ ')')
print('Kind1_ESF.ES: ' + str(round(df['Kind1_ESF.ES'].mean(),3))+ ' (std: ' +str(round(df['Kind1_ESF.ES'].std(),3))+ ')')
print('Kind1_ESF.RR: ' + str(round(df['Kind1_ESF.RR'].mean(),3))+ ' (std: ' +str(round(df['Kind1_ESF.RR'].std(),3))+ ')')
print('Kind1_ESF.SU: ' + str(round(df['Kind1_ESF.SU'].mean(),3))+ ' (std: ' +str(round(df['Kind1_ESF.SU'].std(),3))+ ')')
print('Kind1_ESF.PS: ' + str(round(df['Kind1_ESF.PS'].mean(),3))+ ' (std: ' +str(round(df['Kind1_ESF.PS'].std(),3))+ ')')
print('AlterJahreMonate: ' + str(round(df['AlterJahreMonate'].mean(),3))+ ' (std: ' +str(round(df['AlterJahreMonate'].std(),3))+ ')')
print('SES_2: ' + str(round(df['SES_2'].mean(),3))+ ' (std: ' +str(round(df['SES_2'].std(),3))+ ')')
print('Kind1_CBCL.Tsum: ' + str(round(df['Kind1_CBCL.Tsum'].mean(),3))+ ' (std: ' +str(round(df['Kind1_CBCL.Tsum'].std(),3))+ ')')
print('Geschlecht: ' + str(df[df['Geschlecht']=='weiblich'].shape[0])+' weiblich und '+ str(df[df['Geschlecht']=='männlich'].shape[0]) + ' männlich')
print('Gruppe: ' + str(df[df['Gruppenzugehörigkeit']=='EG'].shape[0])+' EG und '+ str(df[df['Gruppenzugehörigkeit']=='KG'].shape[0]) + ' KG')

#print('Geschlecht: ' + str(df[df['Geschlecht']=='weiblich'][df['Gruppenzugehörigkeit']=="KG"].shape[0]))
#print('Geschlecht: ' + str(df[df['Geschlecht']=='männlich'][df['Gruppenzugehörigkeit']=="KG"].shape[0]))

print('\nAnzahl erhobener Probanden Gießen: ' + str(df[df['Standort']=='Gießen'].shape[0]))
print('Anzahl erhobener Probanden Dortmund: ' + str(df[df['Standort']=='Dortmund'].shape[0]))  
df = Zwischenspeicher.copy()

##### Education and age <a class="anchor" id="eduage"></a>

In [ ]:
#pie chart function, % and total sum are shown
aseg_all = df
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}% ({v:d})'.format(p=pct,v=val)#{p:.2f}% if %is needed
    return my_autopct

#subplot 
fig1, (ax1, ax2) = plt.subplots(1,2, figsize=(18,10)) # 1 row, 2 columns
fig1.tight_layout(w_pad=12)

#plot histogram with fitted norm curve
mu, std = norm.fit(aseg_all['AlterJahreMonate'])
ax2.hist(aseg_all['AlterJahreMonate'], bins=15, alpha=0.6, edgecolor='black',density=True)
ax2.set_title('Age histogram of children')
ax2.set_xlabel('Age')
ax2.set_ylabel('density')
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
ax2.plot(x, p, 'k', linewidth=2)

#plot pie chart
af = aseg_all.groupby(['Kind1_schule'])['Kind1_schule'].count().reset_index(name='count')
ax1.pie(af['count'], labels = af['Kind1_schule'],
       autopct=make_autopct(af['count']),
       wedgeprops = {"edgecolor" : "black", 'linewidth': 0.5, 'antialiased': True})
ax1.set_title("Education")
plt.rc ('font', size = 25) 

dfe = aseg_all[aseg_all['Gruppenzugehörigkeit'] == 'EG']#only EG Group
dfk = aseg_all[aseg_all['Gruppenzugehörigkeit'] == 'KG']#only KG Group

#count and %, children schoolform for EG and KG 
counts = dfe['Kind1_schule'].value_counts()
percs = dfe['Kind1_schule'].value_counts(normalize=True).mul(100).round(2)
print('experimental group\n' + str(pd.concat([counts,percs], axis=1, keys=['count', 'percentage'])))

counts = dfk['Kind1_schule'].value_counts()
percs = dfk['Kind1_schule'].value_counts(normalize=True).mul(100).round(2)
print('\n\ncontrol group\n' + str(pd.concat([counts,percs], axis=1, keys=['count', 'percentage'])))

In [ ]:
from scipy.stats import chi2_contingency

# Daten (N je Stufe): at-risk und control
levels = ["Gymnasium","Grundschule","Gesamtschule","Realschule","Kindergarten"]
at_risk = np.array([10, 9, 7, 1, 1])
control = np.array([17,12, 4, 3, 0])

# Einzelfalldaten erzeugen (Long-Format)
edu, group = [], []
for i, lvl in enumerate(levels):
    edu   += [lvl]*at_risk[i];  group += ["at_risk"]*at_risk[i]
    edu   += [lvl]*control[i];  group += ["control"]*control[i]

df = pd.DataFrame({"education": edu, "group": group})

# Chi-Quadrat-Statistik auf den Originaldaten (nur zur Information)
tab = pd.crosstab(df["education"], df["group"])
chi2_obs = chi2_contingency(tab, correction=False)[0]

# Nicht-konditionaler Permutationstest
rng = np.random.default_rng(123)
B = 100_000  # Anzahl Permutationen (für schnelle Tests z.B. 20_000)
stats = np.empty(B)

g = df["group"].to_numpy()
for b in range(B):
    perm = rng.permutation(g)
    tab_b = pd.crosstab(df["education"], perm)
    stats[b] = chi2_contingency(tab_b, correction=False)[0]

p_perm = (np.sum(stats >= chi2_obs) + 1) / (B + 1)
chi2_obs, p_perm

##### Parental disorder <a class="anchor" id="dis"></a>

In [ ]:
dfe['Patient_Alter_or_moth'] = pd.to_numeric(dfe['Patient_Alter_or_moth'],errors='coerce')#NaN ingore 
dfk['Patient_Alter_or_moth'] = pd.to_numeric(dfk['Patient_Alter_or_moth'],errors='coerce')#NaN ingore 
print('ALter: ' + str(round(dfe['Patient_Alter_or_moth'][dfe['Gruppenzugehörigkeit']=='EG'].mean(),2)) + ' STD: '+  str(round(dfe['Patient_Alter_or_moth'][dfe['Gruppenzugehörigkeit']=='EG'].std(),2)))
print(str(dfe['Patient_geschlecht'][dfe['Gruppenzugehörigkeit']=='EG'].value_counts()))

print('\nMütter der KG Alter: ' + str(round(dfk['Patient_Alter_or_moth'][dfk['Gruppenzugehörigkeit']=='KG'].mean(),2)) + ' STD: '+  str(round(dfk['Patient_Alter_or_moth'][dfk['Gruppenzugehörigkeit']=='KG'].std(),2)))

#plot pie chart with parent illnes
af = df.groupby(['Patient_PrimDiagnoseT1.ICD10'])['Patient_PrimDiagnoseT1.ICD10'].count().reset_index(name='count')

fig, ax = plt.subplots(figsize =(24, 12))
fig.set_facecolor('white')
colors = ['red','green','green','green','orange','orange','purple','beige','beige','violet','violet','yellow','yellow','yellow','turquoise']
explode = (0.1, 0.1, 0.05, 0.05, 0.1, 0.05, 0.02, 0.02, 0.05, 0.05, 0.05, 0.25) 

wedges, texts, autotexts = ax.pie(af['count'], labels = af['Patient_PrimDiagnoseT1.ICD10'],
                                  colors =colors,
                                  autopct=make_autopct(af['count']),
                                  explode =explode,
                                  wedgeprops = {"edgecolor" : "black", 'linewidth': 0.5, 'antialiased': True})

ax.set_title("Parental disorders")
plt.rc ('font', size = 15) 
#ax.legend(wedges, af['Patient_PrimDiagnoseT1.ICD10'],
          #title ='Parental disorders',
          #loc ="center left",
          #bbox_to_anchor =(1.25, 0, 0.5, 1))
   
plt.show() 

print('Mood [affective] disorders & Neurotic, stress-related and somatoform disorders')
print('Delusional disorder vs. Depressive Episode vs. Recurrent depressive disorder vs. Dysthymia vs. Phobic anxiety disorders vs. Other anxiety disorders vs. Reaction to severe stress, and adjustment disorders vs Somatoform disorders')

#plot how many parental diagnosis exist and which one
#fig, axes = plt.subplots(5, 1, figsize=(18, 10))
#fig.set_figwidth(24)
#fig.set_figheight(18)
#fig.tight_layout(w_pad=12)

#sns.countplot(y=df['anzahl_diagT1'],ax=axes[0])
#sns.countplot(y=df['Patient_sek1DiagnoseT1'],alpha=1,ax=axes[1])
#sns.countplot(y=df['Patient_sek2DiagnoseT1'],alpha=1,ax=axes[2])
#sns.countplot(y=df['Patient_sek3DiagnoseT1'],alpha=1,ax=axes[3])
#sns.countplot(y=df['Patient_sek4DiagnoseT1'],alpha=1,ax=axes[4])

print('\n\nNumber of diagnoses:\n' + str(df['anzahl_diagT1'].value_counts()))
print('\n\nSecond diagnosis:\n' + str(df['Patient_sek1DiagnoseT1'].value_counts()))
print('\n\nThird diagnosis:\n' + str(df['Patient_sek2DiagnoseT1'].value_counts()))
print('\n\nFourth diagnosis:\n' + str(df['Patient_sek3DiagnoseT1'].value_counts()))
print('\n\nFifth diagnosis:\n' + str(df['Patient_sek4DiagnoseT1'].value_counts()))

print('\n\nmostly F3 and F4 disorders - depressive episode, Phobic anxiety disorders, anxiety disorders')
 

##### Sample Comparison <a class="anchor" id="samplecom"></a>

In [ ]:
dfttest= df_quest.filter(["Geschwisterkind",'lhilf_avFA_Avg','lhslf1_aFA_Avg', 'lhslf2_aFA_Avg', 'lhslf3_aFA_Avg','lhuf_avgFA_Avg', 'rhilf_avFA_Avg','rhslf1_aFA_Avg', 'rhslf2_aFA_Avg', 'rhslf3_aFA_Avg', 'rhuf_avgFA_Avg','ccgenu_aFA_Avg','rhcbd_avFA_Avg', 'rhcbv_avFA_Avg','lhcbd_avFA_Avg', 'lhcbv_avFA_Avg','Gruppenzugehörigkeit', 'Geschlecht', 'AlterJahreMonate', 'Standort','Kind1_CBCL.Tsum','Kind1_FEEL_KJ.selbst_adaptiv_gesamt','Kind1_FEEL_KJ.selbst_maladaptiv_gesamt','SES_2','Kind1_CBCL.TEXT','Kind1_CBCL.TINT','Kind1_FEEL_KJ.fremd_adaptiv_gesamt','Kind1_FEEL_KJ.fremd_maladaptiv_gesamt', 'Kind1_EFB.NS', 'Kind1_EFB.UR','Kind1_EFB.WS','Kind1_EFB.GW','Kind1_ESF.ES','Kind1_ESF.RR','Kind1_ESF.SU','Kind1_ESF.PS'])
dfttest = dfttest.replace(['männlich','weiblich','Dortmund','Gießen','ja', 'nein','EG','KG'],[1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0])

cols  = ['Kind1_FEEL_KJ.fremd_adaptiv_gesamt','Kind1_FEEL_KJ.fremd_maladaptiv_gesamt','Kind1_FEEL_KJ.selbst_adaptiv_gesamt','Kind1_FEEL_KJ.selbst_maladaptiv_gesamt','Kind1_CBCL.TEXT','Kind1_CBCL.TINT','Kind1_CBCL.Tsum', 'Kind1_EFB.NS', 'Kind1_EFB.UR','Kind1_EFB.WS','Kind1_EFB.GW','Kind1_ESF.ES','Kind1_ESF.RR','Kind1_ESF.SU','Kind1_ESF.PS','SES_2','AlterJahreMonate']
cond = dfttest['Gruppenzugehörigkeit'] == 0

neg_outcome = dfttest.loc[cond, cols]
pos_outcome = dfttest.loc[~cond, cols]

# Welch ttest EG = 11 vs KG = 35
t, p = sc.ttest_ind(neg_outcome, pos_outcome, equal_var=False)
for i, col in enumerate(cols):
    print(f'\t{col}: t = {t[i]:.5f}, with p-value = {p[i]:.5f}')
    

### Frequentist Statistic <a class="anchor" id="frequentist"></a>

##### BoxPlot <a class="anchor" id="boxplot"></a>

In [ ]:
FAhy = ('lhilf_avFA_Avg','lhslf1_aFA_Avg', 'lhslf2_aFA_Avg', 'lhslf3_aFA_Avg','lhuf_avgFA_Avg', 'rhilf_avFA_Avg','rhslf1_aFA_Avg', 'rhslf2_aFA_Avg', 'rhslf3_aFA_Avg', 'rhuf_avgFA_Avg','ccgenu_aFA_Avg','rhcbd_avFA_Avg', 'rhcbv_avFA_Avg','lhcbd_avFA_Avg', 'lhcbv_avFA_Avg')
ADhy = ('lhilf_avAd_Avg','lhslf1_aAd_Avg', 'lhslf2_aAd_Avg','lhslf3_aAd_Avg', 'lhuf_avgAd_Avg', 'rhilf_avAd_Avg','rhslf1_aAd_Avg', 'rhslf2_aAd_Avg', 'rhslf3_aAd_Avg', 'rhuf_avgAd_Avg','ccgenu_aAd_Avg','rhcbd_avAd_Avg', 'rhcbv_avAd_Avg', 'lhcbd_avAd_Avg', 'lhcbv_avAd_Avg')
MDhy = ('lhilf_avMD_Avg','lhslf1_aMD_Avg', 'lhslf2_aMD_Avg','lhslf3_aMD_Avg', 'lhuf_avgMD_Avg', 'rhilf_avMD_Avg', 'rhslf1_aMD_Avg', 'rhslf2_aMD_Avg', 'rhslf3_aMD_Avg', 'rhuf_avgMD_Avg','ccgenu_aMD_Avg','rhcbd_avMD_Avg', 'rhcbv_avMD_Avg', 'lhcbd_avMD_Avg', 'lhcbv_avMD_Avg')
RDhy = ('lhilf_avRD_Avg','lhslf1_aRD_Avg', 'lhslf2_aRD_Avg','lhslf3_aRD_Avg', 'lhuf_avgRD_Avg', 'rhilf_avRD_Avg', 'rhslf1_aRD_Avg', 'rhslf2_aRD_Avg', 'rhslf3_aRD_Avg', 'rhuf_avgRD_Avg','ccgenu_aRD_Avg','rhcbd_avRD_Avg', 'rhcbv_avRD_Avg', 'lhcbd_avRD_Avg', 'lhcbv_avRD_Avg')

fig1, axs = plt.subplots(nrows=3,ncols=5, figsize=(30,11)) # 2 row, 5 columns
sns.set_theme()
ResKS =[]
for ax, response in zip(axs.ravel(),FAhy):
    sns.boxplot(data=final, x='Gruppenzugehörigkeit', y=response, ax=ax)

##### KG vs. EG <a class="anchor" id="vs"></a>

In [ ]:
def format_ols_results(para):
    dff = pd.DataFrame() 
    dff['Param'] = para.params
    dff['tvalues'] = para.tvalues
    dff['pvalues'] = para.pvalues
    dff['rsquared'] = para.rsquared
    dff['rsquared_adj'] = para.rsquared_adj
    return dff

def plotRoi(roi_cols,predictor,Atlas,VolSuf,A,height): 
    n_comparisons =  len(roi_cols) 
    alpha = 0.05
    alpha_corr = 0.05 / n_comparisons
    g = sns.catplot(x='pvalues', y='Areal', kind='bar', height=height, data=A)
    g.set(xscale='log', xlim=(1e-3,2))
    sns.set(rc={'figure.figsize':(30,24)})
    print('Prädiktor '+ predictor+ ' auf '+'Kirterium mit den Covariaten ' + covariates+ ' und p_value_corr: ' + str(alpha_corr))

    for ax in g.axes.flat:
        ax.axvline(alpha, ls='--',c='tomato')
        ax.axvline(alpha_corr, ls='--',c='darkred')
        
#dummycoding
zdf = zdf.replace(['EG','KG'],[1.0,0.0])
df = df.replace(['EG','KG'],[1.0,0.0])
zdf_quest = zdf_quest.replace(['EG','KG'],[1.0,0.0])

#rename for model
zdf_quest = zdf_quest.rename(columns={"Kind1_FEEL_KJ.fremd_adaptiv_gesamt" : "Kind1_FEEL_KJ_fremd_adaptiv_gesamt",
                                      "Kind1_FEEL_KJ.fremd_maladaptiv_gesamt" : "Kind1_FEEL_KJ_fremd_maladaptiv_gesamt",
                                      "Kind1_FEEL_KJ.selbst_adaptiv_gesamt" : "Kind1_FEEL_KJ_selbst_adaptiv_gesamt",
                                      "Kind1_FEEL_KJ.selbst_maladaptiv_gesamt" : "Kind1_FEEL_KJ_selbst_maladaptiv_gesamt",
                                      "Kind1_EFB.NS" : "Kind1_EFB_NS",
                                      "Kind1_EFB.UR" : "Kind1_EFB_UR",
                                      "Kind1_EFB.WS" : "Kind1_EFB_WS",
                                      "Kind1_EFB.GW" : "Kind1_EFB_GW",
                                      "Kind1_ESF.ES" : "Kind1_ESF_ES",
                                      "Kind1_ESF.RR" : "Kind1_ESF_RR",
                                      "Kind1_ESF.SU" : "Kind1_ESF_SU",
                                      "Kind1_ESF.PS" : "Kind1_ESF_PS"})

#colors
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
   
#ROIS
FAhy = ('lhilf_avFA_Avg','lhslf1_aFA_Avg', 'lhslf2_aFA_Avg', 'lhslf3_aFA_Avg','lhuf_avgFA_Avg', 'rhilf_avFA_Avg','rhslf1_aFA_Avg', 'rhslf2_aFA_Avg', 'rhslf3_aFA_Avg', 'rhuf_avgFA_Avg','ccgenu_aFA_Avg', 'rhcbd_avFA_Avg', 'rhcbv_avFA_Avg','lhcbd_avFA_Avg', 'lhcbv_avFA_Avg')
MDhy = ('lhilf_avMD_Avg','lhslf1_aMD_Avg', 'lhslf2_aMD_Avg','lhslf3_aMD_Avg', 'lhuf_avgMD_Avg', 'rhilf_avMD_Avg', 'rhslf1_aMD_Avg', 'rhslf2_aMD_Avg', 'rhslf3_aMD_Avg', 'rhuf_avgMD_Avg','ccgenu_aMD_Avg', 'rhcbd_avMD_Avg', 'rhcbv_avMD_Avg', 'lhcbd_avMD_Avg', 'lhcbv_avMD_Avg')

ADhy = ('lhilf_avAd_Avg','lhslf1_aAd_Avg', 'lhslf2_aAd_Avg','lhslf3_aAd_Avg', 'lhuf_avgAd_Avg', 'rhilf_avAd_Avg','rhslf1_aAd_Avg', 'rhslf2_aAd_Avg', 'rhslf3_aAd_Avg', 'rhuf_avgAd_Avg','ccgenu_aAd_Avg','rhcbd_avAd_Avg', 'rhcbv_avAd_Avg', 'lhcbd_avAd_Avg', 'lhcbv_avAd_Avg')
RDhy = ('lhilf_avRD_Avg','lhslf1_aRD_Avg', 'lhslf2_aRD_Avg','lhslf3_aRD_Avg', 'lhuf_avgRD_Avg', 'rhilf_avRD_Avg', 'rhslf1_aRD_Avg', 'rhslf2_aRD_Avg', 'rhslf3_aRD_Avg', 'rhuf_avgRD_Avg','ccgenu_aRD_Avg','rhcbd_avRD_Avg', 'rhcbv_avRD_Avg', 'lhcbd_avRD_Avg', 'lhcbv_avRD_Avg')

#Hypothesis ROIS
ILF = ('lhilf_avFA_Avg' , 'lhilf_avMD_Avg' , 'rhilf_avFA_Avg', 'rhilf_avMD_Avg')
SLF = ( 'lhslf1_aFA_Avg', 'lhslf2_aFA_Avg', 'lhslf3_aFA_Avg', 'lhslf1_aMD_Avg', 'lhslf2_aMD_Avg','lhslf3_aMD_Avg','rhslf1_aFA_Avg', 'rhslf2_aFA_Avg', 'rhslf3_aFA_Avg', 'rhslf1_aMD_Avg', 'rhslf2_aMD_Avg', 'rhslf3_aMD_Avg')
UF = ('lhuf_avgFA_Avg','lhuf_avgMD_Avg','rhuf_avgFA_Avg', 'rhuf_avgMD_Avg')
CC = ('ccgenu_aFA_Avg', 'ccgenu_aMD_Avg')
Cing = ('rhcbd_avFA_Avg', 'rhcbd_avMD_Avg', 'rhcbv_avFA_Avg', 'rhcbv_avMD_Avg','lhcbd_avFA_Avg', 'lhcbd_avMD_Avg', 'lhcbv_avFA_Avg', 'lhcbv_avMD_Avg')


predictor = ('Gruppenzugehörigkeit')
covariates = ('AlterJahreMonate  + C(Geschlecht,Treatment(reference="männlich"))  + C(Standort,Treatment(reference="Dortmund")) + SES_2 ') 

sns.set_style("darkgrid")

ols_df = pd.DataFrame()
for response in SLF:   #change ILF,SLF,UF,CC,Cing
    res = smf.ols(f"{response} ~ {covariates} + {predictor}", data=zdf).fit()
    res_df = format_ols_results(res)
    res_df['Areal'] = response
    ols_df = pd.concat([res_df,ols_df], axis=0, join='outer')
    
    print('\n\n',color.BOLD, color.RED ,response, color.END, res.summary())
A = ols_df.loc['Gruppenzugehörigkeit']
plotRoi(Cing,predictor,'Tracula White matter','(FA)',A,5) #change vektor

#Multiple Comparison correction Bonferroni Holm
from statsmodels.stats.multitest import multipletests
print('\n Bonferroni-Holm:', multipletests(A['pvalues'],method='holm', is_sorted=False, returnsorted=False))
round(A,3)


In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.stats.stattools import durbin_watson


for response in Cing: #change ILF,SLF,UF,CC,Cing
    res = smf.ols(f"{response} ~ {covariates} + {predictor}", data=zdf).fit()
    print('\n\n' +"\033[1m"+"\033[1;37;40m" ,response,"\033[0m")
    #print(res.summary())
    cls = LinearRegDiagnostic(res)
    vif, fig, ax = cls()
    #print(vif)
    #dw= durbin_watson(res.resid)
    #print(f"\n\n Durbin-Watson: {dw}")#1.5-2.5 good Unkorreliertheit der Residuen bzw. Fehler
    
    #print(OLSInfluence(res).summary_table(float_fmt='%6.3f'))

##### Correlation parenting skill <a class="anchor" id="corr"></a>

In [ ]:
ILF_ = df.filter(['lhilf_avFA_Avg' , 'lhilf_avMD_Avg' , 'rhilf_avFA_Avg', 'rhilf_avMD_Avg','Kind1_EFB.GW','Gruppenzugehörigkeit'])
SLF_ = df.filter(['lhslf1_aFA_Avg', 'lhslf2_aFA_Avg', 'lhslf3_aFA_Avg', 'lhslf1_aMD_Avg', 'lhslf2_aMD_Avg','lhslf3_aMD_Avg','rhslf1_aFA_Avg', 'rhslf2_aFA_Avg', 'rhslf3_aFA_Avg', 'rhslf1_aMD_Avg', 'rhslf2_aMD_Avg', 'rhslf3_aMD_Avg','Kind1_EFB.GW','Gruppenzugehörigkeit'])
UF_ = df.filter(['lhuf_avgFA_Avg','lhuf_avgMD_Avg','rhuf_avgFA_Avg', 'rhuf_avgMD_Avg','Kind1_EFB.GW','Gruppenzugehörigkeit'])
CC_ = df.filter(['ccgenu_aFA_Avg', 'ccgenu_aMD_Avg','Kind1_EFB.GW','Gruppenzugehörigkeit'])
Cing_ = df.filter(['rhcbd_avFA_Avg', 'rhcbd_avMD_Avg', 'rhcbv_avFA_Avg', 'rhcbv_avMD_Avg','lhcbd_avFA_Avg', 'lhcbd_avMD_Avg', 'lhcbv_avFA_Avg', 'lhcbv_avMD_Avg','Kind1_EFB.GW','Gruppenzugehörigkeit'])

sns.set_theme(style="ticks")
sns.pairplot(SLF_, hue="Gruppenzugehörigkeit") #change ILF_;SLF_;UF_;CC_;Cing_

In [ ]:
zdf_quest_without = zdf_quest.drop(index=34)

predictor = ('Gruppenzugehörigkeit+Kind1_EFB_GW')
covariates = ('SES_2 + AlterJahreMonate  + C(Geschlecht,Treatment(reference="männlich"))  + C(Standort,Treatment(reference="Dortmund"))') 

sns.set_style("darkgrid")
ols_df = pd.DataFrame()
for response in Cing:   #change ILF,SLF,UF,CC,Cing
    res = smf.ols(f"{response} ~ {covariates} + {predictor}", data=zdf_quest).fit()
    res_df = format_ols_results(res)
    res_df['Areal'] = response
    ols_df = pd.concat([res_df,ols_df], axis=0, join='outer')
    
    print('\n\n',color.BOLD, color.RED ,response, color.END, res.summary())
A = ols_df.loc['Kind1_EFB_GW']
#plotRoi(Cing,predictor,'Tracula White matter','(FA)',A,5) #change vektor


#Multiple Comparison correction Bonferroni Holm
from statsmodels.stats.multitest import multipletests
print('\n Bonferroni-Holm:', multipletests(A['pvalues'],method='holm', is_sorted=False, returnsorted=False))
round(A,3)

In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.stats.stattools import durbin_watson

for response in SLF: #change ILF,SLF,UF,CC,Cing
    res = smf.ols(f"{response} ~ {covariates} + {predictor}", data=zdf_quest_without).fit()
    print('\n\n' +"\033[1m"+"\033[1;37;40m" ,response,"\033[0m")
    #print(res.summary())
    cls = LinearRegDiagnostic(res)
    vif, fig, ax = cls()
    #print(vif)
    #dw= durbin_watson(res.resid)
    #print(f"\n\n Durbin-Watson: {dw}")#1.5-2.5 good Unkorreliertheit der Residuen bzw. Fehler
    
    #print(OLSInfluence(res).summary_table(float_fmt='%6.3f'))
    
#Looks like row 17 has much influence in the regression it corresponde to index 33 


##### Correlation parenting stress <a class="anchor" id="corrstress"></a>

In [ ]:
ILF_ = df.filter(['lhilf_avFA_Avg' , 'lhilf_avMD_Avg' , 'rhilf_avFA_Avg', 'rhilf_avMD_Avg','Kind1_ESF.ES','Kind1_ESF.RR','Kind1_ESF.SU','Kind1_ESF.PS','Gruppenzugehörigkeit'])
SLF_ = df.filter(['lhslf1_aFA_Avg', 'lhslf2_aFA_Avg', 'lhslf3_aFA_Avg', 'lhslf1_aMD_Avg', 'lhslf2_aMD_Avg','lhslf3_aMD_Avg','rhslf1_aFA_Avg', 'rhslf2_aFA_Avg', 'rhslf3_aFA_Avg', 'rhslf1_aMD_Avg', 'rhslf2_aMD_Avg', 'rhslf3_aMD_Avg','Kind1_ESF.ES','Kind1_ESF.RR','Kind1_ESF.SU','Kind1_ESF.PS','Gruppenzugehörigkeit'])
UF_ = df.filter(['lhuf_avgFA_Avg','lhuf_avgMD_Avg','rhuf_avgFA_Avg', 'rhuf_avgMD_Avg','Kind1_ESF.ES','Kind1_ESF.RR','Kind1_ESF.SU','Kind1_ESF.PS','Gruppenzugehörigkeit'])
CC_ = df.filter(['ccgenu_aFA_Avg', 'ccgenu_aMD_Avg','Kind1_ESF.ES','Kind1_ESF.RR','Kind1_ESF.SU','Kind1_ESF.PS','Gruppenzugehörigkeit'])
Cing_ = df.filter(['rhcbd_avFA_Avg', 'rhcbd_avMD_Avg', 'rhcbv_avFA_Avg', 'rhcbv_avMD_Avg','lhcbd_avFA_Avg', 'lhcbd_avMD_Avg', 'lhcbv_avFA_Avg', 'lhcbv_avMD_Avg','Kind1_ESF.ES','Kind1_ESF.RR','Kind1_ESF.SU','Kind1_ESF.PS','Gruppenzugehörigkeit'])

sns.set_theme(style="ticks")
sns.pairplot(ILF_, hue="Gruppenzugehörigkeit")

In [ ]:
##predictor 

#"Gruppenzugehörigkeit+Kind1_ESF_ES"
#"Gruppenzugehörigkeit+Kind1_ESF_RR"
#"Gruppenzugehörigkeit+Kind1_ESF_SU"
#"Gruppenzugehörigkeit+Kind1_ESF_PS"

covariates = ('SES_2 + AlterJahreMonate  + C(Geschlecht,Treatment(reference="männlich"))  + C(Standort,Treatment(reference="Dortmund"))')
sns.set_style("darkgrid")

ols_df_ES = pd.DataFrame()
ols_df_RR = pd.DataFrame()
ols_df_SU = pd.DataFrame()
ols_df_PS = pd.DataFrame()
for response in Cing:   #change ILF,SLF,UF,CC,Cing
    res_ES = smf.ols(f"{response} ~ {covariates} + Gruppenzugehörigkeit+Kind1_ESF_ES", data=zdf_quest).fit()
    res_df_ES = format_ols_results(res_ES)
    res_df_ES['Areal'] = response
    ols_df_ES = pd.concat([res_df_ES,ols_df_ES], axis=0, join='outer')
    print('\n\n',color.BOLD, color.RED ,response, color.END, res_ES.summary())
    
    res_RR = smf.ols(f"{response} ~ {covariates} + Gruppenzugehörigkeit+Kind1_ESF_RR", data=zdf_quest).fit()
    res_df_RR = format_ols_results(res_RR)
    res_df_RR['Areal'] = response
    ols_df_RR = pd.concat([res_df_RR,ols_df_RR], axis=0, join='outer')
    print('\n\n',color.BOLD, color.RED ,response, color.END, res_RR.summary())
    
    res_SU = smf.ols(f"{response} ~ {covariates} + Gruppenzugehörigkeit+Kind1_ESF_SU", data=zdf_quest).fit()
    res_df_SU = format_ols_results(res_SU)
    res_df_SU['Areal'] = response
    ols_df_SU = pd.concat([res_df_SU,ols_df_SU], axis=0, join='outer')
    print('\n\n',color.BOLD, color.RED ,response, color.END, res_SU.summary())
    
    res_PS = smf.ols(f"{response} ~ {covariates} + Gruppenzugehörigkeit+Kind1_ESF_PS", data=zdf_quest).fit()
    res_df_PS = format_ols_results(res_PS)
    res_df_PS['Areal'] = response
    ols_df_PS = pd.concat([res_df_PS,ols_df_PS], axis=0, join='outer')
    print('\n\n',color.BOLD, color.RED ,response, color.END, res_PS.summary())
ES = ols_df_ES.loc['Kind1_ESF_ES']
RR = ols_df_RR.loc['Kind1_ESF_RR']
SU = ols_df_SU.loc['Kind1_ESF_SU']
PS = ols_df_PS.loc['Kind1_ESF_PS']
#plotRoi(Cing,predictor,'Tracula White matter','(FA)',A,5) #change vektor


#Multiple Comparison correction Bonferroni Holm
#from statsmodels.stats.multitest import multipletests
#print('\n Bonferroni-Holm:', multipletests(A['pvalues'],method='holm', is_sorted=False, returnsorted=False))
#print(round(ES,3),'\n')
#print(round(RR,3),'\n')
#print(round(SU,3),'\n')
#print(round(PS,3),'\n')
insgesamt_ESF = pd.concat([ES,RR,SU,PS])
#Multiple Comparison correction Bonferroni Holm
from statsmodels.stats.multitest import multipletests
print('\n Bonferroni-Holm:', multipletests(insgesamt_ESF['pvalues'],method='holm', is_sorted=False, returnsorted=False))
insgesamt_ESF

In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.stats.stattools import durbin_watson

predictor = ('Gruppenzugehörigkeit+Kind1_ESF_PS')
#Gruppenzugehörigkeit+Kind1_ESF_ES
#Gruppenzugehörigkeit+Kind1_ESF_RR
#Gruppenzugehörigkeit+Kind1_ESF_SU
#Gruppenzugehörigkeit+Kind1_ESF_PS

for response in Cing: #change ILF,SLF,UF,CC,Cing
    res = smf.ols(f"{response} ~ {covariates} + {predictor}", data=zdf_quest).fit()
    print('\n\n' +"\033[1m"+"\033[1;37;40m" ,response,"\033[0m")
    #print(res.summary())
    cls = LinearRegDiagnostic(res)
    vif, fig, ax = cls()
    #print(vif)
    #dw= durbin_watson(res.resid)
    #print(f"\n\n Durbin-Watson: {dw}")#1.5-2.5 good Unkorreliertheit der Residuen bzw. Fehler
    
    #print(OLSInfluence(res).summary_table(float_fmt='%6.3f'))

##### Correlation Emotionregulation <a class="anchor" id="emotion"></a>

In [ ]:
ILF_ = df.filter(['lhilf_avFA_Avg' , 'lhilf_avMD_Avg' , 'rhilf_avFA_Avg', 'rhilf_avMD_Avg','Kind1_FEEL_KJ.fremd_adaptiv_gesamt','Kind1_FEEL_KJ.fremd_maladaptiv_gesamt','Kind1_FEEL_KJ.selbst_adaptiv_gesamt','Kind1_FEEL_KJ.selbst_maladaptiv_gesamt','Gruppenzugehörigkeit'])
SLF_ = df.filter(['lhslf1_aFA_Avg', 'lhslf2_aFA_Avg', 'lhslf3_aFA_Avg', 'lhslf1_aMD_Avg', 'lhslf2_aMD_Avg','lhslf3_aMD_Avg','rhslf1_aFA_Avg', 'rhslf2_aFA_Avg', 'rhslf3_aFA_Avg', 'rhslf1_aMD_Avg', 'rhslf2_aMD_Avg', 'rhslf3_aMD_Avg','Kind1_FEEL_KJ.fremd_adaptiv_gesamt','Kind1_FEEL_KJ.fremd_maladaptiv_gesamt','Kind1_FEEL_KJ.selbst_adaptiv_gesamt','Kind1_FEEL_KJ.selbst_maladaptiv_gesamt','Gruppenzugehörigkeit'])
UF_ = df.filter(['lhuf_avgFA_Avg','lhuf_avgMD_Avg','rhuf_avgFA_Avg', 'rhuf_avgMD_Avg','Kind1_FEEL_KJ.fremd_adaptiv_gesamt','Kind1_FEEL_KJ.fremd_maladaptiv_gesamt','Kind1_FEEL_KJ.selbst_adaptiv_gesamt','Kind1_FEEL_KJ.selbst_maladaptiv_gesamt','Gruppenzugehörigkeit'])
CC_ = df.filter(['ccgenu_aFA_Avg', 'ccgenu_aMD_Avg','Kind1_FEEL_KJ.fremd_adaptiv_gesamt','Kind1_FEEL_KJ.fremd_maladaptiv_gesamt','Kind1_FEEL_KJ.selbst_adaptiv_gesamt','Kind1_FEEL_KJ.selbst_maladaptiv_gesamt','Gruppenzugehörigkeit'])
Cing_ = df.filter(['rhcbd_avFA_Avg', 'rhcbd_avMD_Avg', 'rhcbv_avFA_Avg', 'rhcbv_avMD_Avg','lhcbd_avFA_Avg', 'lhcbd_avMD_Avg', 'lhcbv_avFA_Avg', 'lhcbv_avMD_Avg','Kind1_FEEL_KJ.fremd_adaptiv_gesamt','Kind1_FEEL_KJ.fremd_maladaptiv_gesamt','Kind1_FEEL_KJ.selbst_adaptiv_gesamt','Kind1_FEEL_KJ.selbst_maladaptiv_gesamt','Gruppenzugehörigkeit'])

sns.set_theme(style="ticks")
sns.pairplot(ILF_, hue="Gruppenzugehörigkeit")

In [ ]:
covariates = ('AlterJahreMonate  + C(Geschlecht,Treatment(reference="männlich"))  + C(Standort,Treatment(reference="Dortmund"))')
sns.set_style("darkgrid")

ols_df_fremd_ada = pd.DataFrame()
ols_df_fremd_mal = pd.DataFrame()
ols_df_selbst_ada = pd.DataFrame()
ols_df_selbst_mal = pd.DataFrame()
for response in Cing:   #change ILF,SLF,UF,CC,Cing
    res_fremd_ada = smf.ols(f"{response} ~ {covariates} + Gruppenzugehörigkeit+Kind1_FEEL_KJ_fremd_adaptiv_gesamt", data=zdf_quest).fit()
    res_df_fremd_ada = format_ols_results(res_fremd_ada)
    res_df_fremd_ada['Areal'] = response
    ols_df_fremd_ada = pd.concat([res_df_fremd_ada,ols_df_fremd_ada], axis=0, join='outer')
    print('\n\n',color.BOLD, color.RED ,response, color.END, res_fremd_ada.summary())
    
    res_fremd_mal = smf.ols(f"{response} ~ {covariates} + Gruppenzugehörigkeit+Kind1_FEEL_KJ_fremd_maladaptiv_gesamt", data=zdf_quest).fit()
    res_df_fremd_mal = format_ols_results(res_fremd_mal)
    res_df_fremd_mal['Areal'] = response
    ols_df_fremd_mal = pd.concat([res_df_fremd_mal,ols_df_fremd_mal], axis=0, join='outer')
    print('\n\n',color.BOLD, color.RED ,response, color.END, res_fremd_mal.summary())
    
    res_selbst_ada = smf.ols(f"{response} ~ {covariates} + Gruppenzugehörigkeit+Kind1_FEEL_KJ_selbst_adaptiv_gesamt", data=zdf_quest).fit()
    res_df_selbst_ada = format_ols_results(res_selbst_ada)
    res_df_selbst_ada['Areal'] = response
    ols_df_selbst_ada = pd.concat([res_df_selbst_ada,ols_df_selbst_ada], axis=0, join='outer')
    print('\n\n',color.BOLD, color.RED ,response, color.END, res_selbst_ada.summary())
    
    res_selbst_mal = smf.ols(f"{response} ~ {covariates} + Gruppenzugehörigkeit+Kind1_FEEL_KJ_selbst_maladaptiv_gesamt", data=zdf_quest).fit()
    res_df_selbst_mal = format_ols_results(res_selbst_mal)
    res_df_selbst_mal['Areal'] = response
    ols_df_selbst_mal = pd.concat([res_df_selbst_mal,ols_df_selbst_mal], axis=0, join='outer')
    print('\n\n',color.BOLD, color.RED ,response, color.END, res_selbst_mal.summary())
fremd_ada = ols_df_fremd_ada.loc['Kind1_FEEL_KJ_fremd_adaptiv_gesamt']
fremd_mal = ols_df_fremd_mal.loc['Kind1_FEEL_KJ_fremd_maladaptiv_gesamt']
selbst_ada = ols_df_selbst_ada.loc['Kind1_FEEL_KJ_selbst_adaptiv_gesamt']
selbst_mal = ols_df_selbst_mal.loc['Kind1_FEEL_KJ_selbst_maladaptiv_gesamt']
#plotRoi(Cing,predictor,'Tracula White matter','(FA)',A,5) #change vektor

#Multiple Comparison correction Bonferroni Holm
#from statsmodels.stats.multitest import multipletests
#print('\n Bonferroni-Holm:', multipletests(A['pvalues'],method='holm', is_sorted=False, returnsorted=False))
#print(round(fremd_ada,3),'\n')
#print(round(fremd_mal,3),'\n')
#print(round(selbst_ada,3),'\n')
#print(round(selbst_mal,3),'\n')
insgesamt_FEEL = pd.concat([fremd_ada,fremd_mal,selbst_ada,selbst_mal])
#Multiple Comparison correction Bonferroni Holm
from statsmodels.stats.multitest import multipletests
print('\n Bonferroni-Holm:', multipletests(insgesamt_FEEL['pvalues'],method='holm', is_sorted=False, returnsorted=False))
insgesamt_FEEL

In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.stats.stattools import durbin_watson

predictor = ('Gruppenzugehörigkeit+Kind1_FEEL_KJ_selbst_adaptiv_gesamt')
#Gruppenzugehörigkeit+Kind1_FEEL_KJ_fremd_adaptiv_gesamt
#Gruppenzugehörigkeit+Kind1_FEEL_KJ_fremd_maladaptiv_gesamt
#Gruppenzugehörigkeit+Kind1_FEEL_KJ_selbst_adaptiv_gesamt
#Gruppenzugehörigkeit+Kind1_FEEL_KJ_selbst_maladaptiv_gesamt

for response in Cing: #change ILF,SLF,UF,CC,Cing
    res = smf.ols(f"{response} ~ {covariates} + {predictor}", data=zdf_quest).fit()
    print('\n\n' +"\033[1m"+"\033[1;37;40m" ,response,"\033[0m")
    #print(res.summary())
    cls = LinearRegDiagnostic(res)
    vif, fig, ax = cls()
    #print(vif)
    #dw= durbin_watson(res.resid)
    #print(f"\n\n Durbin-Watson: {dw}")#1.5-2.5 good Unkorreliertheit der Residuen bzw. Fehler
    
    #print(OLSInfluence(res).summary_table(float_fmt='%6.3f'))

### Correlation Heatmaps <a class="anchor" id="heat"></a>

In [ ]:
df = zdf_quest.filter(["Kind1_FEEL_KJ_fremd_adaptiv_gesamt","Kind1_FEEL_KJ_fremd_maladaptiv_gesamt","Kind1_FEEL_KJ_selbst_adaptiv_gesamt","Kind1_FEEL_KJ_selbst_maladaptiv_gesamt","Kind1_EFB_GW","Kind1_ESF_ES","Kind1_ESF_RR","Kind1_ESF_SU","Kind1_ESF_PS",
"SES_2","AlterJahreMonate","Geschlecht","Standort","Gruppenzugehörigkeit",'lhslf1_aFA_Avg','lhslf2_aFA_Avg','lhslf3_aFA_Avg','lhuf_avgFA_Avg','lhilf_avFA_Avg','ccgenu_aFA_Avg','lhcbd_avFA_Avg','lhcbv_avFA_Avg','rhslf1_aFA_Avg','rhslf2_aFA_Avg',
'rhslf3_aFA_Avg', 'rhuf_avgFA_Avg','rhilf_avFA_Avg','rhcbd_avFA_Avg','rhcbv_avFA_Avg','lhslf1_aMD_Avg', 'lhslf2_aMD_Avg','lhslf3_aMD_Avg', 'rhslf1_aMD_Avg', 'rhslf2_aMD_Avg', 'rhslf3_aMD_Avg','lhuf_avgMD_Avg', 'rhuf_avgMD_Avg',
'lhilf_avMD_Avg' , 'rhilf_avMD_Avg','ccgenu_aMD_Avg','rhcbd_avMD_Avg', 'rhcbv_avMD_Avg', 'lhcbd_avMD_Avg', 'lhcbv_avMD_Avg'])


df = df.rename(columns={              'lhslf1_aMD_Avg': "l_SLF1_MD",
                                      'lhslf2_aMD_Avg' : "l_SLF2_MD",
                                      'lhslf3_aMD_Avg' : "l_SLF3_MD",
                                      'lhuf_avgMD_Avg' : "l_UF_MD",
                                      'lhilf_avMD_Avg' : "l_ILF_MD",
                                      'ccgenu_aMD_Avg' : "CCgenu_MD",
                                      'lhcbd_avMD_Avg' : "l_Cing_dorsal_MD",
                                      'lhcbv_avMD_Avg' : 'l_Cing_ventral_MD',
                                      'rhslf1_aMD_Avg' : 'r_SLF1_MD',
                                      'rhslf2_aMD_Avg' : 'r_SLF2_MD',
                                      'rhslf3_aMD_Avg' : 'r_SLF3_MD',
                                      'rhuf_avgMD_Avg' : 'r_UF_MD',
                                      'rhilf_avMD_Avg' : 'r_ILF_MD',
                                      'rhcbd_avMD_Avg' : 'r_Cing_dorsal_MD',
                                      'rhcbv_avMD_Avg' : 'r_Cing_ventral_MD',              
                                      'lhslf1_aFA_Avg' : "l_SLF1_FA",
                                      'lhslf2_aFA_Avg' : "l_SLF2_FA",
                                      'lhslf3_aFA_Avg' : "l_SLF3_FA",
                                      'lhuf_avgFA_Avg' : "l_UF_FA",
                                      'lhilf_avFA_Avg' : "l_ILF_FA",
                                      'ccgenu_aFA_Avg' : "CCgenu_FA",
                                      'lhcbd_avFA_Avg' : "l_Cing_dorsal_FA",
                                      'lhcbv_avFA_Avg' : 'l_Cing_ventral_FA',
                                      'rhslf1_aFA_Avg' : 'r_SLF1_FA',
                                      'rhslf2_aFA_Avg' : 'r_SLF2_FA',
                                      'rhslf3_aFA_Avg' : 'r_SLF3_FA',
                                      'rhuf_avgFA_Avg' : 'r_UF_FA',
                                      'rhilf_avFA_Avg' : 'r_ILF_FA',
                                      'rhcbd_avFA_Avg' : 'r_Cing_dorsal_FA',
                                      'rhcbv_avFA_Avg' : 'r_Cing_ventral_FA',
                                      "Kind1_FEEL_KJ_fremd_adaptiv_gesamt" : "FEEL_KJ_extern_adaptiv",
                                      "Kind1_FEEL_KJ_fremd_maladaptiv_gesamt" : "FEEL_KJ_extern_maladaptiv",
                                      "Kind1_FEEL_KJ_selbst_adaptiv_gesamt" : "FEEL_KJ_self_adaptiv",
                                      "Kind1_FEEL_KJ_selbst_maladaptiv_gesamt" : "FEEL_KJ_self_maladaptiv",
                                      "Kind1_EFB_GW" : "EFB_GW",
                                      "Kind1_ESF_ES" : "ESF_ES",
                                      "Kind1_ESF_RR" : "ESF_RR",
                                      "Kind1_ESF_SU" : "ESF_SU",
                                      "Kind1_ESF_PS" : "ESF_PS",
                                      "SES_2" : "SES",
                                      "AlterJahreMonate" : "Age",
                                      "Geschlecht" : "Sex",
                                      "Standort" : "Scanner_Location",
                                      "Gruppenzugehörigkeit" : "Group",})

df = df.replace(['männlich','weiblich','Dortmund','Gießen','ja', 'nein','EG','KG'],[1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0])


In [ ]:
#FA
#dataframe = df.filter(["Group","Scanner_Location","Sex","Age","SES","ESF_PS","ESF_SU","ESF_RR","ESF_ES","EFB_GW","FEEL_KJ_self_maladaptiv","FEEL_KJ_self_adaptiv","FEEL_KJ_extern_maladaptiv","FEEL_KJ_extern_adaptiv",'r_Cing_ventral_FA','r_Cing_dorsal_FA','r_ILF_FA','r_UF_FA','r_SLF3_FA','r_SLF2_FA','r_SLF1_FA','l_Cing_ventral_FA',"l_Cing_dorsal_FA","CCgenu_FA","l_ILF_FA","l_UF_FA","l_SLF3_FA","l_SLF2_FA","l_SLF1_FA"])
#MD
dataframe = df.filter(["Group","Scanner Location","Sex","Age","SES","ESF_PS","ESF_SU","ESF_RR","ESF_ES","EFB_GW","FEEL_KJ_self_maladaptiv","FEEL_KJ_self_adaptiv","FEEL_KJ_extern_maladaptiv","FEEL_KJ_extern_adaptiv",'r_Cing_ventral_MD','r_Cing_dorsal_MD','r_ILF_MD','r_UF_MD','r_SLF3_MD','r_SLF2_MD','r_SLF1_MD','l_Cing_ventral_MD',"l_Cing_dorsal_MD","CCgenu_MD","l_ILF_MD","l_UF_MD","l_SLF3_MD","l_SLF2_MD","l_SLF1_MD"])

def corr_sig(dataframe=None):
    p_matrix = np.zeros(shape=(dataframe.shape[1],dataframe.shape[1]))
    for col in dataframe.columns:
        for col2 in dataframe.drop(col,axis=1).columns:
            _ , p = sc.pearsonr(dataframe[col],dataframe[col2])#pearsonr
            p_matrix[dataframe.columns.to_list().index(col),dataframe.columns.to_list().index(col2)] = p
    return p_matrix

Bonferroni=0.05
  
corr = dataframe.corr()                            # get correlation
p_values = corr_sig(dataframe)                     # get p-Value
mask = np.invert(np.tril(p_values <= Bonferroni))    # mask - only get significant corr
#plot_cor_matrix(corr,mask)
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 12)) 
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5,annot=True, annot_kws={"size": 8.5})

### Bayesian Statistics - R Kernel <a class="anchor" id="bayes"></a>

In [ ]:
#################################
####Change Kernel to R Kernel####
#################################


setwd("") #set path to data
if (!require("pacman")) install.packages("pacman")
 pacman::p_load(bayestestR, dplyr,fastDummies, ggplot2,ggpubr,BayesFactor,rstanarm,varBF,bayesplot, see, performance, tidyr,rcompanion,lsr)
 #check if packages are available from github install_github(...) - by using devtool packages or remotes package

dfR <- read.table("data.csv",sep = ",",fileEncoding="UTF-8", header=T,dec = "." )
zdfR <- dfR %>% mutate(across(where(is.double), scale))
zdfR <- dummy_cols(zdfR, select_columns = c('Gruppenzugehörigkeit','Standort','Geschlecht')) 

# Subset df by removing rows with NA values in specific columns
dfR_quest <- dfR %>%
  drop_na(Kind1_ESF.ES, 
          Kind1_FEEL_KJ.selbst_maladaptiv_gesamt, 
          Kind1_EFB.WS, 
          Kind1_FEEL_KJ.fremd_adaptiv_gesamt)

zdfR_quest <- zdfR %>%
  drop_na(Kind1_ESF.ES, 
          Kind1_FEEL_KJ.selbst_maladaptiv_gesamt, 
          Kind1_EFB.WS, 
          Kind1_FEEL_KJ.fremd_adaptiv_gesamt)



##### Sample Comparison with R <a class="anchor" id="samplecomR"></a>


In [ ]:
#normal distributed -  Boxplot, QQPlot,histo with fitted curve, density distribution compare the observed distribution to what we would expect 
#Kind1_CBCL.Tsum has missing values 
defaultW <- getOption("warn") 
options(warn = -1) 

    for(i in c("AlterJahreMonate", "SES_2","Kind1_CBCL.Tsum")) {
        a <- ggplot(dfR, aes(x = Gruppenzugehörigkeit, y = .data[[i]])) +
            geom_boxplot(fill="lightgray") + 
            geom_dotplot(binaxis = "y", stackdir = "center", dotsize = 0.2,binwidth = 0.5)

        b <- ggqqplot(dfR, x = i,
                color = "Gruppenzugehörigkeit", 
                palette = c("#0073C2FF", "#FC4E07"),
                ggtheme = theme_pubclean())+
                ylab(i)+
                xlab("Theoretical Quantiles")
             

        c <- gghistogram(
                        dfR, x = i, y = "..density..",
                        add = "mean", rug = TRUE, bins=30,
                        fill = "Gruppenzugehörigkeit", palette = c("#00AFBB", "#E7B800"),
                        add_density = TRUE)
        d <- ggdensity(zdfR, x = i, fill = "lightgray", title = i) +
                        coord_cartesian() +
                        stat_overlay_normal_density(color = "red", linetype = "dashed")     

        options(repr.plot.width=15, repr.plot.height=10)
        g <-     (ggarrange(a,b,c,d, 
            labels = c("A", "B","C","D"),
            ncol = 2, nrow = 2))
        print(annotate_figure(g, top = text_grob(paste0(i), 
               color = "red", face = "bold", size = 14)))
    }

options(warn = defaultW)  

In [ ]:
#ttest; dfR_qust has only 11 EG and 36 KG 
#when using dfR only the NA in the specific variable will be ignored meaning every ttest hast a different sample size in EG and KG -> dfR_quest is the sample removed every NA in the questionaires of interest


#    for(i in c('Kind1_FEEL_KJ.fremd_adaptiv_gesamt','Kind1_FEEL_KJ.fremd_maladaptiv_gesamt','Kind1_FEEL_KJ.selbst_adaptiv_gesamt','Kind1_FEEL_KJ.selbst_maladaptiv_gesamt','Kind1_CBCL.TEXT','Kind1_CBCL.TINT','Kind1_CBCL.Tsum', 'Kind1_EFB.NS', 'Kind1_EFB.UR','Kind1_EFB.WS','Kind1_EFB.GW','Kind1_ESF.ES','Kind1_ESF.RR','Kind1_ESF.SU','Kind1_ESF.PS','SES_2','AlterJahreMonate')) {
#        result[[i]] <- (t.test(dfR_quest[[i]] ~ dfR_quest$Gruppenzugehörigkeit, var.equal = FALSE, alternative = "two.sided"))
#        print(i);print(result[[i]])
#    }
result <- list()
cohen <- list()
    for(i in c("AlterJahreMonate", "SES_2","Kind1_CBCL.Tsum")) {
        result[[i]] <- (t.test(dfR[[i]] ~ dfR$Gruppenzugehörigkeit, var.equal = FALSE, alternative = "two.sided"))
        cohen[[i]] <- cohensD(dfR[[i]] ~ dfR$Gruppenzugehörigkeit)
        print(i);print(result[[i]]);print(cohen[[i]]);cat("\n\n")
    }

#Chi² or Fisher(better) for eduaction between groups
kreuztabelle <- xtabs (~ zdfR$Gruppenzugehörigkeit + zdfR$Kind1_schule)
n <- sum(kreuztabelle)
erwartete_häufigkeiten <- outer (rowSums(kreuztabelle), colSums(kreuztabelle)) / n
erwartete_häufigkeiten
#chisq.test(zdfR$Gruppenzugehörigkeit, zdfR$Kind1_schule)
fisher.test(zdfR$Gruppenzugehörigkeit, zdfR$Kind1_schule)

kreuztabelle <- xtabs (~ zdfR$Gruppenzugehörigkeit + zdfR$Geschlecht)
n <- sum(kreuztabelle)
erwartete_häufigkeiten <- outer (rowSums(kreuztabelle), colSums(kreuztabelle)) / n
erwartete_häufigkeiten
chisq.test(zdfR$Gruppenzugehörigkeit, zdfR$Geschlecht)
cohenW(zdfR$Gruppenzugehörigkeit, zdfR$Geschlecht)



In [ ]:
res <- list()
#correlations
for (i in c("Kind1_EFB.GW","Kind1_ESF.ES","Kind1_ESF.RR","Kind1_ESF.SU","Kind1_ESF.PS",'Kind1_FEEL_KJ.selbst_adaptiv_gesamt','Kind1_FEEL_KJ.fremd_adaptiv_gesamt','Kind1_FEEL_KJ.fremd_maladaptiv_gesamt','Kind1_FEEL_KJ.selbst_adaptiv_gesamt','Kind1_FEEL_KJ.selbst_maladaptiv_gesamt')) {
    res[[i]] <- (cor.test(zdfR_quest[[i]],zdfR_quest$SES_2, method = "pearson"))
    print(i); print(res[[i]])
}


##### KG vs. EG Bayes <a class="anchor" id="bayeseg"></a>

In [ ]:
#Hypothesis ROIS
ILF = c('lhilf_avFA_Avg' , 'lhilf_avMD_Avg' , 'rhilf_avFA_Avg', 'rhilf_avMD_Avg')
SLF = c( 'lhslf1_aFA_Avg', 'lhslf2_aFA_Avg', 'lhslf3_aFA_Avg', 'lhslf1_aMD_Avg', 'lhslf2_aMD_Avg','lhslf3_aMD_Avg','rhslf1_aFA_Avg', 'rhslf2_aFA_Avg', 'rhslf3_aFA_Avg', 'rhslf1_aMD_Avg', 'rhslf2_aMD_Avg', 'rhslf3_aMD_Avg')
UF = c('lhuf_avgFA_Avg','lhuf_avgMD_Avg','rhuf_avgFA_Avg', 'rhuf_avgMD_Avg')
CC = c('ccgenu_aFA_Avg', 'ccgenu_aMD_Avg')
Cing = c('rhcbd_avFA_Avg', 'rhcbd_avMD_Avg', 'rhcbv_avFA_Avg', 'rhcbv_avMD_Avg','lhcbd_avFA_Avg', 'lhcbd_avMD_Avg', 'lhcbv_avFA_Avg', 'lhcbv_avMD_Avg')

#Prior
locationpa = 0 # priors (0,1/sqrt(2)); (0,1); (0,sqrt(2)); (0,10)
scalepa = 1

model=list()
for(i in ILF) { #change vektor ILF;SLF;UF;CC;Cing
    set.seed(123)
    model[[i]] <- stan_glm(paste0(i, " ~ SES_2 + AlterJahreMonate  + Geschlecht_männlich +  Standort_Dortmund + Gruppenzugehörigkeit_EG ")  , data = zdfR, #change SES_2 
        prior=normal(locationpa,scalepa,autoscale=FALSE),
        prior_intercept=normal(locationpa,scalepa,autoscale=FALSE),  # priors (0,1/sqrt(2)); (0,1); (0,sqrt(2)); (0,10)
        refresh = 0,
        iter = 15000, #15000 iterations-5000warmups*5chains = 50000
        warmup =5000,
        chains =5)
    
    print(i);cat("\n");
    #print(tidyMCMC(model[[i]], conf.int = TRUE, conf.method = "HPDinterval"))
    #print(check_collinearity(model[[i]]))
    print(describe_posterior(model[[i]], priors=TRUE, centrality="all", ci_method="hdi",test=c("p_direction", "rope","bf","pd","ps"), diagnostic="all",ci=0.95,rope_ci=1,rope_range = "default"))
    #print(prior_summary(model[[i]]))
    #print(bf_rope(model[[i]])) #BFrope
    #print(point_estimate(model[[i]]))
    cat("\n\n")
}


In [ ]:
#check visually Posteriors
#change used ROI vektor to the same as above

for(ROI in ILF) { #change vektor ILF;SLF;UF;CC;Cing
  posterior <- as.matrix(model[[ROI]])
  plot_title <- ggtitle("Posterior distributions",
                        "with medians and 95% intervals")
  a <- mcmc_areas(posterior,
            pars = c("SES_2", "AlterJahreMonate", "Geschlecht_männlich","Gruppenzugehörigkeit_EG"),
            prob = 0.95) + plot_title

  b <- plot(bayesfactor_parameters(model[[ROI]],parameter="Gruppenzugehörigkeit_EG")) +
    scale_color_material() +
    scale_fill_material()    #results in a plot presenting the prior and posterior distributions for parameter. When a point null was tested, two dots represent the density of the null at the value - the ratio of their heights is the value of the Savage-Dickey Bayes factor
                  
  options(repr.plot.width=19, repr.plot.height=8)
  g <- ggarrange(a,b, ncol=2,nrow=1)
  print(annotate_figure(g, top = text_grob(paste0(ROI), 
          color = "red", face = "bold", size = 14)))

}

In [ ]:
#change used ROI vektor to the same as above
for(i in SLF) { #change vektor ILF;SLF;UF;CC;Cing
  resid = resid(model[[i]])
  fit = fitted(model[[i]])
  sresid = resid/sd(resid)

  options(repr.plot.width=22, repr.plot.height=18)
  #non-linearity, unequal error variances, and outliers (e.g. residual vs fitted plot)
  a <- ggplot(data=NULL,mapping=aes(x=fit,y=resid)) +
              geom_point(shape=1) +
              geom_hline(yintercept=0,linetype="dashed") +
              geom_smooth(color="red",linetype = "dashed",linewidth=0.5)+
              ylab("Residuals")+
              xlab("Fitted valus")+
              ggtitle("Residual vs. Fitted")


  #qqPlot of residuals (should be normal distributed)
  b <- ggplot(data=NULL, aes(sample = sresid)) +
              ggtitle("Normal Q-Q")+  
              ylab("Standardized residuals")+
              xlab("Theoretical Quantiles")+
              geom_qq(shape=1) +
              geom_qq_line(linetype = "dashed",color="red")

  #Posterior predictive check (makes model sense to explain data)
  c <- pp_check(model[[i]], nreps=100) + xlab(paste0(i)) + ggtitle("Posterior predictive check") + theme(plot.title = element_text(hjust = 0.06))

  #this compares the posterior estimate for each parameter against the associated prior. 
  #If the spread of the priors is small relative to the posterior, then it is likely that the priors are too influential.
  d <- posterior_vs_prior(model[[i]], color_by = "vs", group_by = TRUE, 
                          facet_args = list(scales = "free_y")) + ggtitle("Posterior vs. Prior")

  color_scheme_set("mix-blue-red")
  #take a look at the posteriors for each chain and the trace 
  #Trace plots show no evidence that the chains have not reasonably traversed the entire multidimensional parameter space
  e <- mcmc_combo(model[[i]],
                  combo = c("dens_overlay", "trace"), 
                  gg_theme = ggplot2::theme_gray()) 

  #autocorrelation check 
  f <- mcmc_acf(model[[i]])
 
  g <- (ggarrange(a,b,c,d,e,f, 
                  labels = c("A", "B","C","D","E","F"),
                  ncol = 2, nrow = 3))

  print(annotate_figure(g, top = text_grob(paste0(i), 
                        color = "red", face = "bold", size = 14)))

}

##### Correlation parenting skill Bayes <a class="anchor" id="corrbayes"></a>

In [ ]:
#Hypothesis ROIS
ILF = c('lhilf_avFA_Avg' , 'lhilf_avMD_Avg' , 'rhilf_avFA_Avg', 'rhilf_avMD_Avg')
SLF = c( 'lhslf1_aFA_Avg', 'lhslf2_aFA_Avg', 'lhslf3_aFA_Avg', 'lhslf1_aMD_Avg', 'lhslf2_aMD_Avg','lhslf3_aMD_Avg','rhslf1_aFA_Avg', 'rhslf2_aFA_Avg', 'rhslf3_aFA_Avg', 'rhslf1_aMD_Avg', 'rhslf2_aMD_Avg', 'rhslf3_aMD_Avg')
UF = c('lhuf_avgFA_Avg','lhuf_avgMD_Avg','rhuf_avgFA_Avg', 'rhuf_avgMD_Avg')
CC = c('ccgenu_aFA_Avg', 'ccgenu_aMD_Avg')
Cing = c('rhcbd_avFA_Avg', 'rhcbd_avMD_Avg', 'rhcbv_avFA_Avg', 'rhcbv_avMD_Avg','lhcbd_avFA_Avg', 'lhcbd_avMD_Avg', 'lhcbv_avFA_Avg', 'lhcbv_avMD_Avg')

# Remove rows where the 'ID' column is 'sub-070' the one making problems above
zdfR_quest_without <- zdfR_quest[zdfR_quest$ID != 'sub-070',]

#Prior
locationpa = 0 # priors (0,1/sqrt(2)); (0,1); (0,sqrt(2)); (0,10)
scalepa = 1

model=list()
for(i in SLF) { #change vektor ILF;SLF;UF;CC;Cing
    set.seed(123)
    model[[i]] <- stan_glm(paste0(i, " ~ AlterJahreMonate  + Geschlecht_männlich +  Standort_Dortmund + Gruppenzugehörigkeit_EG + Kind1_EFB.GW ")  , data = zdfR_quest, #change SES_2 
        prior=normal(locationpa,scalepa,autoscale=FALSE),
        prior_intercept=normal(locationpa,scalepa,autoscale=FALSE),  # priors (0,1/sqrt(2)); (0,1); (0,sqrt(2)); (0,10)
        refresh = 0,
        iter = 15000, #15000 iterations-5000warmups*5chains = 50000
        warmup =5000,
        chains =5)
    
    print(i);cat("\n");
    #print(tidyMCMC(model[[i]], conf.int = TRUE, conf.method = "HPDinterval"))
    #print(check_collinearity(model[[i]]))
    print(describe_posterior(model[[i]], priors=TRUE, centrality="all", ci_method="hdi",test=c("p_direction", "rope","bf","pd","ps"), diagnostic="all",ci=0.95,rope_ci=1,rope_range = "default"))
    #print(prior_summary(model[[i]]))
    #print(bf_rope(model[[i]])) #BFrope
    #print(point_estimate(model[[i]]))
    cat("\n\n")
}

In [ ]:
#check visually Posteriors
#change used ROI vektor to the same as above

for(ROI in ILF) { #change vektor ILF;SLF;UF;CC;Cing
  posterior <- as.matrix(model[[ROI]])
  plot_title <- ggtitle("Posterior distributions",
                        "with medians and 95% intervals")
  a <- mcmc_areas(posterior,
            pars = c("SES_2", "AlterJahreMonate", "Geschlecht_männlich","Gruppenzugehörigkeit_EG", "Kind1_EFB.GW"),
            prob = 0.95) + plot_title

  b <- plot(bayesfactor_parameters(model[[ROI]],parameter="Kind1_EFB.GW")) +
    scale_color_material() +
    scale_fill_material()    #results in a plot presenting the prior and posterior distributions for parameter. When a point null was tested, two dots represent the density of the null at the value - the ratio of their heights is the value of the Savage-Dickey Bayes factor
                  
  options(repr.plot.width=19, repr.plot.height=8)
  g <- ggarrange(a,b, ncol=2,nrow=1)
  print(annotate_figure(g, top = text_grob(paste0(ROI), 
          color = "red", face = "bold", size = 14)))

}

In [ ]:
#change used ROI vektor to the same as above
for(i in ILF) { #change vektor ILF;SLF;UF;CC;Cing
  resid = resid(model[[i]])
  fit = fitted(model[[i]])
  sresid = resid/sd(resid)

  options(repr.plot.width=22, repr.plot.height=18)
  #non-linearity, unequal error variances, and outliers (e.g. residual vs fitted plot)
  a <- ggplot(data=NULL,mapping=aes(x=fit,y=resid)) +
              geom_point(shape=1) +
              geom_hline(yintercept=0,linetype="dashed") +
              geom_smooth(color="red",linetype = "dashed",linewidth=0.5)+
              ylab("Residuals")+
              xlab("Fitted valus")+
              ggtitle("Residual vs. Fitted")


  #qqPlot of residuals (should be normal distributed)
  b <- ggplot(data=NULL, aes(sample = sresid)) +
              ggtitle("Normal Q-Q")+  
              ylab("Standardized residuals")+
              xlab("Theoretical Quantiles")+
              geom_qq(shape=1) +
              geom_qq_line(linetype = "dashed",color="red")

  #Posterior predictive check (makes model sense to explain data)
  c <- pp_check(model[[i]], nreps=100) + xlab(paste0(i)) + ggtitle("Posterior predictive check") + theme(plot.title = element_text(hjust = 0.06))

  #this compares the posterior estimate for each parameter against the associated prior. 
  #If the spread of the priors is small relative to the posterior, then it is likely that the priors are too influential.
  d <- posterior_vs_prior(model[[i]], color_by = "vs", group_by = TRUE, 
                          facet_args = list(scales = "free_y")) + ggtitle("Posterior vs. Prior")

  color_scheme_set("mix-blue-red")
  #take a look at the posteriors for each chain and the trace 
  #Trace plots show no evidence that the chains have not reasonably traversed the entire multidimensional parameter space
  e <- mcmc_combo(model[[i]],
                  combo = c("dens_overlay", "trace"), 
                  gg_theme = ggplot2::theme_gray()) 

  #autocorrelation check 
  f <- mcmc_acf(model[[i]])
 
  g <- (ggarrange(a,b,c,d,e,f, 
                  labels = c("A", "B","C","D","E","F"),
                  ncol = 2, nrow = 3))

  print(annotate_figure(g, top = text_grob(paste0(i), 
                        color = "red", face = "bold", size = 14)))

}

##### Correlation parenting stress Bayes <a class="anchor" id="corrstressbayes"></a>

In [ ]:
#Hypothesis ROIS
ILF = c('lhilf_avFA_Avg' , 'lhilf_avMD_Avg' , 'rhilf_avFA_Avg', 'rhilf_avMD_Avg')
SLF = c( 'lhslf1_aFA_Avg', 'lhslf2_aFA_Avg', 'lhslf3_aFA_Avg', 'lhslf1_aMD_Avg', 'lhslf2_aMD_Avg','lhslf3_aMD_Avg','rhslf1_aFA_Avg', 'rhslf2_aFA_Avg', 'rhslf3_aFA_Avg', 'rhslf1_aMD_Avg', 'rhslf2_aMD_Avg', 'rhslf3_aMD_Avg')
UF = c('lhuf_avgFA_Avg','lhuf_avgMD_Avg','rhuf_avgFA_Avg', 'rhuf_avgMD_Avg')
CC = c('ccgenu_aFA_Avg', 'ccgenu_aMD_Avg')
Cing = c('rhcbd_avFA_Avg', 'rhcbd_avMD_Avg', 'rhcbv_avFA_Avg', 'rhcbv_avMD_Avg','lhcbd_avFA_Avg', 'lhcbd_avMD_Avg', 'lhcbv_avFA_Avg', 'lhcbv_avMD_Avg')

# Remove rows where the 'ID' column is 'sub-070' the one making problems above
zdfR_quest_without <- zdfR_quest[zdfR_quest$ID != 'sub-070',]


#ESF - Predictor of interest
ESF = c("Kind1_ESF.PS") #'Kind1_ESF.ES','Kind1_ESF.RR','Kind1_ESF.SU','Kind1_ESF.PS'

#Prior
locationpa = 0 # priors (0,1/sqrt(2)); (0,1); (0,sqrt(2)); (0,10)
scalepa = 1

model=list()
for(i in ILF) { #change vektor ILF;SLF;UF;CC;Cing
    set.seed(123)
    model[[i]] <- stan_glm(paste0(i, " ~ SES_2 + AlterJahreMonate  + Geschlecht_männlich +  Standort_Dortmund + Gruppenzugehörigkeit_EG + ", ESF)  , data = zdfR_quest, #change SES_2
        prior=normal(locationpa,scalepa,autoscale=FALSE),
        prior_intercept=normal(locationpa,scalepa,autoscale=FALSE),  # priors (0,1/sqrt(2)); (0,1); (0,sqrt(2)); (0,10)
        refresh = 0,
        iter = 15000, #15000 iterations-5000warmups*5chains = 50000
        warmup =5000,
        chains =5)
    
    print(i);cat("\n");
    #print(tidyMCMC(model[[i]], conf.int = TRUE, conf.method = "HPDinterval"))
    #print(check_collinearity(model[[i]]))
    print(describe_posterior(model[[i]], priors=TRUE, centrality="all", ci_method="hdi",test=c("p_direction", "rope","bf","pd","ps"), diagnostic="all",ci=0.95,rope_ci=1,rope_range = "default"))
    #print(prior_summary(model[[i]]))
    #print(bf_rope(model[[i]])) #BFrope
    #print(point_estimate(model[[i]]))
    cat("\n\n")
}

In [ ]:
#check visually Posteriors
#change used ROI vektor to the same as above

for(ROI in ILF) { #change vektor ILF;SLF;UF;CC;Cing
  posterior <- as.matrix(model[[ROI]])
  plot_title <- ggtitle("Posterior distributions",
                        "with medians and 95% intervals")
  a <- mcmc_areas(posterior,
            pars = c("SES_2", "AlterJahreMonate", "Geschlecht_männlich","Gruppenzugehörigkeit_EG", ESF),
            prob = 0.95) + plot_title

  b <- plot(bayesfactor_parameters(model[[ROI]],parameter=ESF)) +
    scale_color_material() +
    scale_fill_material()    #results in a plot presenting the prior and posterior distributions for parameter. When a point null was tested, two dots represent the density of the null at the value - the ratio of their heights is the value of the Savage-Dickey Bayes factor
                  
  options(repr.plot.width=19, repr.plot.height=8)
  g <- ggarrange(a,b, ncol=2,nrow=1)
  print(annotate_figure(g, top = text_grob(paste0(ROI), 
          color = "red", face = "bold", size = 14)))

}

In [ ]:
#change used ROI vektor to the same as above
for(i in ILF) { #change vektor ILF;SLF;UF;CC;Cing
  resid = resid(model[[i]])
  fit = fitted(model[[i]])
  sresid = resid/sd(resid)

  options(repr.plot.width=22, repr.plot.height=18)
  #non-linearity, unequal error variances, and outliers (e.g. residual vs fitted plot)
  a <- ggplot(data=NULL,mapping=aes(x=fit,y=resid)) +
              geom_point(shape=1) +
              geom_hline(yintercept=0,linetype="dashed") +
              geom_smooth(color="red",linetype = "dashed",linewidth=0.5)+
              ylab("Residuals")+
              xlab("Fitted valus")+
              ggtitle("Residual vs. Fitted")


  #qqPlot of residuals (should be normal distributed)
  b <- ggplot(data=NULL, aes(sample = sresid)) +
              ggtitle("Normal Q-Q")+  
              ylab("Standardized residuals")+
              xlab("Theoretical Quantiles")+
              geom_qq(shape=1) +
              geom_qq_line(linetype = "dashed",color="red")

  #Posterior predictive check (makes model sense to explain data)
  c <- pp_check(model[[i]], nreps=100) + xlab(paste0(i)) + ggtitle("Posterior predictive check") + theme(plot.title = element_text(hjust = 0.06))

  #this compares the posterior estimate for each parameter against the associated prior. 
  #If the spread of the priors is small relative to the posterior, then it is likely that the priors are too influential.
  d <- posterior_vs_prior(model[[i]], color_by = "vs", group_by = TRUE, 
                          facet_args = list(scales = "free_y")) + ggtitle("Posterior vs. Prior")

  color_scheme_set("mix-blue-red")
  #take a look at the posteriors for each chain and the trace 
  #Trace plots show no evidence that the chains have not reasonably traversed the entire multidimensional parameter space
  e <- mcmc_combo(model[[i]],
                  combo = c("dens_overlay", "trace"), 
                  gg_theme = ggplot2::theme_gray()) 

  #autocorrelation check 
  f <- mcmc_acf(model[[i]])
 
  g <- (ggarrange(a,b,c,d,e,f, 
                  labels = c("A", "B","C","D","E","F"),
                  ncol = 2, nrow = 3))

  print(annotate_figure(g, top = text_grob(paste0(i), 
                        color = "red", face = "bold", size = 14)))

}

##### Correlation Emotionregulation Bayes <a class="anchor" id="emotionbayes"></a>

In [ ]:
#Hypothesis ROIS
ILF = c('lhilf_avFA_Avg' , 'lhilf_avMD_Avg' , 'rhilf_avFA_Avg', 'rhilf_avMD_Avg')
SLF = c( 'lhslf1_aFA_Avg', 'lhslf2_aFA_Avg', 'lhslf3_aFA_Avg', 'lhslf1_aMD_Avg', 'lhslf2_aMD_Avg','lhslf3_aMD_Avg','rhslf1_aFA_Avg', 'rhslf2_aFA_Avg', 'rhslf3_aFA_Avg', 'rhslf1_aMD_Avg', 'rhslf2_aMD_Avg', 'rhslf3_aMD_Avg')
UF = c('lhuf_avgFA_Avg','lhuf_avgMD_Avg','rhuf_avgFA_Avg', 'rhuf_avgMD_Avg')
CC = c('ccgenu_aFA_Avg', 'ccgenu_aMD_Avg')
Cing = c('rhcbd_avFA_Avg', 'rhcbd_avMD_Avg', 'rhcbv_avFA_Avg', 'rhcbv_avMD_Avg','lhcbd_avFA_Avg', 'lhcbd_avMD_Avg', 'lhcbv_avFA_Avg', 'lhcbv_avMD_Avg')

# Remove rows where the 'ID' column is 'sub-070' the one making problems above
zdfR_quest_without <- zdfR_quest[zdfR_quest$ID != 'sub-070',]


#Emot - Predictor of interest
Emot = c('Kind1_FEEL_KJ.selbst_adaptiv_gesamt') #'Kind1_FEEL_KJ.fremd_adaptiv_gesamt','Kind1_FEEL_KJ.fremd_maladaptiv_gesamt','Kind1_FEEL_KJ.selbst_adaptiv_gesamt','Kind1_FEEL_KJ.selbst_maladaptiv_gesamt'

#Prior
locationpa = 0 # priors (0,1/sqrt(2)); (0,1); (0,sqrt(2)); (0,10)
scalepa = 1

model=list()
for(i in ILF) { #change vektor ILF;SLF;UF;CC;Cing
    set.seed(123)
    model[[i]] <- stan_glm(paste0(i, " ~ SES_2 + AlterJahreMonate  + Geschlecht_männlich +  Standort_Dortmund + Gruppenzugehörigkeit_EG + ", Emot)  , data = zdfR_quest, #change SES_2
        prior=normal(locationpa,scalepa,autoscale=FALSE),
        prior_intercept=normal(locationpa,scalepa,autoscale=FALSE),  # priors (0,1/sqrt(2)); (0,1); (0,sqrt(2)); (0,10)
        refresh = 0,
        iter = 15000, #15000 iterations-5000warmups*5chains = 50000
        warmup =5000,
        chains =5)
    
    print(i);cat("\n");
    #print(tidyMCMC(model[[i]], conf.int = TRUE, conf.method = "HPDinterval"))
    #print(check_collinearity(model[[i]]))
    print(describe_posterior(model[[i]], priors=TRUE, centrality="all", ci_method="hdi",test=c("p_direction", "rope","bf","pd","ps"), diagnostic="all",ci=0.95,rope_ci=1,rope_range = "default"))
    #print(prior_summary(model[[i]]))
    #print(bf_rope(model[[i]])) #BFrope
    #print(point_estimate(model[[i]]))
    cat("\n\n")
}

In [ ]:
#check visually Posteriors
#change used ROI vektor to the same as above

for(ROI in ILF) { #change vektor ILF;SLF;UF;CC;Cing
  posterior <- as.matrix(model[[ROI]])
  plot_title <- ggtitle("Posterior distributions",
                        "with medians and 95% intervals")
  a <- mcmc_areas(posterior,
            pars = c("SES_2", "AlterJahreMonate", "Geschlecht_männlich","Gruppenzugehörigkeit_EG", Emot),
            prob = 0.95) + plot_title

  b <- plot(bayesfactor_parameters(model[[ROI]],parameter=Emot)) +
    scale_color_material() +
    scale_fill_material()    #results in a plot presenting the prior and posterior distributions for parameter. When a point null was tested, two dots represent the density of the null at the value - the ratio of their heights is the value of the Savage-Dickey Bayes factor
                  
  options(repr.plot.width=19, repr.plot.height=8)
  g <- ggarrange(a,b, ncol=2,nrow=1)
  print(annotate_figure(g, top = text_grob(paste0(ROI), 
          color = "red", face = "bold", size = 14)))

}

In [ ]:
#change used ROI vektor to the same as above
for(i in ILF) { #change vektor ILF;SLF;UF;CC;Cing
  resid = resid(model[[i]])
  fit = fitted(model[[i]])
  sresid = resid/sd(resid)

  options(repr.plot.width=22, repr.plot.height=18)
  #non-linearity, unequal error variances, and outliers (e.g. residual vs fitted plot)
  a <- ggplot(data=NULL,mapping=aes(x=fit,y=resid)) +
              geom_point(shape=1) +
              geom_hline(yintercept=0,linetype="dashed") +
              geom_smooth(color="red",linetype = "dashed",linewidth=0.5)+
              ylab("Residuals")+
              xlab("Fitted valus")+
              ggtitle("Residual vs. Fitted")


  #qqPlot of residuals (should be normal distributed)
  b <- ggplot(data=NULL, aes(sample = sresid)) +
              ggtitle("Normal Q-Q")+  
              ylab("Standardized residuals")+
              xlab("Theoretical Quantiles")+
              geom_qq(shape=1) +
              geom_qq_line(linetype = "dashed",color="red")

  #Posterior predictive check (makes model sense to explain data)
  c <- pp_check(model[[i]], nreps=100) + xlab(paste0(i)) + ggtitle("Posterior predictive check") + theme(plot.title = element_text(hjust = 0.06))

  #this compares the posterior estimate for each parameter against the associated prior. 
  #If the spread of the priors is small relative to the posterior, then it is likely that the priors are too influential.
  d <- posterior_vs_prior(model[[i]], color_by = "vs", group_by = TRUE, 
                          facet_args = list(scales = "free_y")) + ggtitle("Posterior vs. Prior")

  color_scheme_set("mix-blue-red")
  #take a look at the posteriors for each chain and the trace 
  #Trace plots show no evidence that the chains have not reasonably traversed the entire multidimensional parameter space
  e <- mcmc_combo(model[[i]],
                  combo = c("dens_overlay", "trace"), 
                  gg_theme = ggplot2::theme_gray()) 

  #autocorrelation check 
  f <- mcmc_acf(model[[i]])
 
  g <- (ggarrange(a,b,c,d,e,f, 
                  labels = c("A", "B","C","D","E","F"),
                  ncol = 2, nrow = 3))

  print(annotate_figure(g, top = text_grob(paste0(i), 
                        color = "red", face = "bold", size = 14)))

}